In [1]:
import numpy as np
np.random.seed(0)
import tensorflow as tf
tf.set_random_seed(0)
PYTHONHASHSEED=0

In [2]:
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.75
set_session(tf.Session(config=config))

Using TensorFlow backend.


In [3]:
import os
import gc
import math
import glob
import datetime
import copy
import nibabel as nib
import tensorflow as tf
from keras import metrics
from keras import backend as K
from random import *
from keras.models import Input, Model, load_model
from keras.layers import Conv2D, Concatenate, MaxPooling2D
from keras.layers import UpSampling2D, Dropout, BatchNormalization
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.optimizers import rmsprop
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.advanced_activations import LeakyReLU, PReLU

'''
U-Net: Convolutional Networks for Biomedical Image Segmentation
(https://arxiv.org/abs/1505.04597)
---
img_shape: (height, width, channels)
out_ch: number of output channels
start_ch: number of channels of the first conv
depth: zero indexed depth of the U-structure
inc_rate: rate at which the conv channels will increase
activation: activation function after convolutions
dropout: amount of dropout in the contracting part
batchnorm: adds Batch Normalization if true
maxpool: use strided conv instead of maxpooling if false
upconv: use transposed conv instead of upsamping + conv if false
residual: add residual connections around each conv block if true
'''

'\nU-Net: Convolutional Networks for Biomedical Image Segmentation\n(https://arxiv.org/abs/1505.04597)\n---\nimg_shape: (height, width, channels)\nout_ch: number of output channels\nstart_ch: number of channels of the first conv\ndepth: zero indexed depth of the U-structure\ninc_rate: rate at which the conv channels will increase\nactivation: activation function after convolutions\ndropout: amount of dropout in the contracting part\nbatchnorm: adds Batch Normalization if true\nmaxpool: use strided conv instead of maxpooling if false\nupconv: use transposed conv instead of upsamping + conv if false\nresidual: add residual connections around each conv block if true\n'

In [4]:
def conv_block(m, dim, acti, bn, res, do=0):
    if acti != 'leaky_relu':
        n = Conv2D(dim, 3, activation=acti, padding='same')(m)
        n = BatchNormalization()(n) if bn else n
        n = Dropout(do)(n) if do else n
        n = Conv2D(dim, 3, activation=acti, padding='same')(n)
        n = BatchNormalization()(n) if bn else n
        return Concatenate()([m, n]) if res else n
    else:
        n = Conv2D(dim, 3, activation='linear', padding='same')(m)
        n = LeakyReLU(alpha=.001)(n)
        n = BatchNormalization()(n) if bn else n
        n = Dropout(do)(n) if do else n
        n = Conv2D(dim, 3, activation='linear', padding='same')(n)
        n = LeakyReLU(alpha=.001)(n)
        n = BatchNormalization()(n) if bn else n
        return Concatenate()([m, n]) if res else n

def level_block(m, dim, depth, inc, acti, do, bn, mp, up, res):
	if depth > 0:
		n = conv_block(m, dim, acti, bn, res)
		m = MaxPooling2D()(n) if mp else Conv2D(dim, 3, strides=2, padding='same')(n)
		m = level_block(m, int(inc*dim), depth-1, inc, acti, do, bn, mp, up, res)
		if up:
			m = UpSampling2D()(m)
			m = Conv2D(dim, 2, activation=acti, padding='same')(m)
		else:
			m = Conv2DTranspose(dim, 3, strides=2, activation=acti, padding='same')(m)
		n = Concatenate()([n, m])
		m = conv_block(n, dim, acti, bn, res)
	else:
		m = conv_block(m, dim, acti, bn, res, do)
	return m

def UNet(img_shape, out_ch=1, start_ch=64, depth=4, inc_rate=2., activation='relu', 
		 dropout=0.5, batchnorm=False, maxpool=True, upconv=True, residual=False):
	i = Input(shape=img_shape)
	o = level_block(i, start_ch, depth, inc_rate, activation,
                    dropout, batchnorm, maxpool, upconv, residual)
	o = Conv2D(out_ch, 1, activation='sigmoid')(o)
	return Model(inputs=i, outputs=o)

In [5]:
def mean_squared_error_1e12(y_true, y_pred):
    return K.mean(K.square(y_pred - y_true), axis=-1)*1e12

def mean_squared_error_1e6(y_true, y_pred):
    return K.mean(K.square(y_pred - y_true), axis=-1)*1e6

def mean_absolute_error_1e6(y_true, y_pred):
    return K.mean(K.abs(y_pred - y_true), axis=-1)*1e6

def psnr(y_true, y_pred):
#     return -10.0*K.log(1.0/(K.mean(K.square(y_pred - y_true))))/K.log(10.0)
    mse = K.mean(K.square(y_pred - y_true))
    return (20 - 10 * K.log(mse)/K.log(10.0))*1e3

def mse1e12_weighted(y_true, y_pred):
    diff = np.dot(K.square(y_pred - y_true), y_pred)
    loss = K.mean(diff, axis=-1)
    return loss*1e12

In [6]:
list_patient = ['mre003', 'mre005', 'mre006', 'mre008', 'mre010',
                'mre011', 'mre012', 'mre013', 'mre014', 'mre016',
                'mre017', 'mre018', 'mre019', 'mre022', 'mre023',
                'mre024', 'mre025', 'mre026', 'mre027', 'mre028']

In [7]:
def output_dataset(filename, list_train, list_val, LOOCV):

    file_name = filename + "dataset.txt"
    now = datetime.datetime.now()
    date = now.strftime("%Y-%m-%d")
    
    with open(file_name, "w") as text_file:
        print("Date: " , date , file=text_file)
        print("Produced by Winston Chen", file=text_file)
        print("Number of the training set: ", 14, file=text_file)
        print("Number of the testing set: ", 5, file=text_file)
        print(' ', file=text_file)
        print("The training set names:", file=text_file)
        for i in list_train:
            print(i, file=text_file)
        print(' ', file=text_file)
        print("The validation set names:", file=text_file)
        for i in list_val:
            print(i, file=text_file)
        print(' ', file=text_file)
        print("The testing set names:", file=text_file)
        print(list_patient[LOOCV], file=text_file)

In [8]:
def train_a_UNet(depth=4, epochs=10, loss='mse1e6', n_fliter=64,
                 optimizer='Adam', lr=1e-4, batch_size='5',
                 LOOCV = 0, model_type='PET-only'):
    
    if model_type == 'PET-only':
        slice_x = 3
    if model_type == 'PET-MR':
        slice_x = 4
    if model_type == 'MR-only':
        slice_x = 1
    
    
    #----------------------------------------------Configurations----------------------------------------------#
    
    
    # logs
    log_path = '.\\logs\\'+ model_type + '_' + str(LOOCV) + "\\"
    if not os.path.exists(log_path):
        os.makedirs(log_path)
    tensorboard = TensorBoard(log_dir=log_path, batch_size=batch_size,
                              write_graph=True, write_grads=True,
                              write_images=True)
    
    # set traininig configurations
    conf = {"image_shape" : (192, 192, slice_x), "out_channel" : 1, "filter" : n_fliter, "depth" : depth,
            "inc_rate" : 2, "activation" : 'relu', "dropout" : True, "batchnorm" : True, "maxpool" : True,
            "upconv" : True, "residual" : True, "shuffle" : True, "augmentation" : True,
            "learning_rate" : 1e-5, "decay" : 0.0, "epsilon" : 1e-8, "beta_1" : 0.9, "beta_2" : 0.999,
            "validation_split" : 0.2632, "batch_size" : batch_size, "epochs" : epochs,
            "loss": loss, "metric" : "mse", "optimizer" : optimizer, "LOOCV" : LOOCV, "model_type" : model_type}
    np.save(log_path + model_type + '_' + str(LOOCV) + '_info.npy', conf) 
    
    
    # set augmentation configurations
    conf_a = {"rotation_range": 15, "shear_range" : 10,
              "width_shift_range" : 0.33, "height_shift_range" : 0.33, "zoom_range" : 0.33,
              "horizontal_flip" : True, "vertical_flip" : True, "fill_mode" : 'nearest',
              "seed" : 314, "batch_size" : conf["batch_size"]}
    np.save(log_path + model_type + '_' + str(LOOCV) + '_aug.npy', conf_a)

    
    # checkpoint
    check_path= '.\\training_models\\' + model_type + '_' + str(LOOCV) + '\\'
    if not os.path.exists(check_path):
        os.makedirs(check_path)    
    check_path= check_path + 'model.hdf5' # _{epoch:03d}_{val_loss:.4f}
    checkpoint1 = ModelCheckpoint(check_path, monitor='val_psnr',
                                  verbose=1, save_best_only=True, mode='max')
#     checkpoint2 = ModelCheckpoint(check_path, period=100)
    callbacks_list = [checkpoint1, tensorboard]
    
    
    #----------------------------------------------Create Model----------------------------------------------#
    
    
    # build up the model
    model = UNet(img_shape=conf["image_shape"], out_ch=conf["out_channel"],
                 start_ch=conf["filter"], depth=conf["depth"],
                 inc_rate=conf["inc_rate"], activation=conf["activation"],
                 dropout=conf["dropout"], batchnorm=conf["batchnorm"],
                 maxpool=conf["maxpool"], upconv=conf["upconv"],
                 residual=conf["residual"])

    
    # Adam optimizer
    if conf["optimizer"] == 'Adam':
        opt = Adam(lr=conf["learning_rate"], decay=conf["decay"],
                   epsilon=conf["epsilon"], beta_1=conf["beta_1"], beta_2=conf["beta_2"])
    if conf["loss"] == 'mse1e6':
        loss = mean_squared_error_1e6

    
    # load dataset [1780, 192, 192, 1]
    datapath = ".//dataset//npy//"
    list_number = np.asarray(range(20))
    list_number = np.delete(list_number, LOOCV)
#     np.random.shuffle(list_number)
    list_train = []
    list_val = []
    for i in range(5):
        list_val.append(list_patient[list_number[i]])
    for i in range(14):
        list_train.append(list_patient[list_number[i+4]])
    output_dataset(log_path + model_type + '_' + str(LOOCV), list_train, list_val, LOOCV)
    
    x_test = np.zeros((89*1,192,192,slice_x), dtype=np.float32)
    y_test = np.zeros((89*1,192,192,1), dtype=np.float32)
    x_val = np.zeros((89*5,192,192,slice_x), dtype=np.float32)
    y_val = np.zeros((89*5,192,192,1), dtype=np.float32)
    x_train = np.zeros((89*14,192,192,slice_x), dtype=np.float32)
    y_train = np.zeros((89*14,192,192,1), dtype=np.float32)
    
    # MR-only model
    if model_type == 'MR-only':
        amp_x = 3000
        amp_y = 14000
        temp_x = np.load(datapath+list_patient[LOOCV]+'_water_data.npy')
        temp_y = np.load(datapath+list_patient[LOOCV]+'_5min_data.npy')
        for idx in range(89):
            x_test[idx,:,:,:] = temp_x[:,:,idx].reshape((1,192,192,slice_x)) / amp_x
            y_test[idx,:,:,:] = temp_y[:,:,idx].reshape((1,192,192,1)) / amp_y
        
        idx_x = 0
        idx_y = 0
        for patient in list_val:
            name_x = datapath+patient+'_water_data.npy'
            name_y = datapath+patient+'_5min_data.npy'
            temp_x = np.load(name_x)
            temp_y = np.load(name_y)
            for idx in range(89):
                x_val[idx_x,:,:,:] = temp_x[:,:,idx].reshape((1,192,192,slice_x)) / amp_x
                y_val[idx_y,:,:,:] = temp_y[:,:,idx].reshape((1,192,192,1)) / amp_y
                idx_x = idx_x + 1
                idx_y = idx_y + 1    
        x_val = x_val
        y_val = y_val

    
        idx_x = 0
        idx_y = 0
        for patient in list_train:
            name_x = datapath+patient+'_water_data.npy'
            name_y = datapath+patient+'_5min_data.npy'
            temp_x = np.load(name_x)
            temp_y = np.load(name_y)
            for idx in range(89):
                x_train[idx_x,:,:,:] = temp_x[:,:,idx].reshape((1,192,192,slice_x)) / amp_x
                y_train[idx_y,:,:,:] = temp_y[:,:,idx].reshape((1,192,192,1)) / amp_y
                idx_x = idx_x + 1
                idx_y = idx_y + 1  
        
    
    # PET-only model
    if model_type == 'PET-only':
        amp_x = 14000
        amp_y = 14000
        temp_x = np.load(datapath+list_patient[LOOCV]+'_1min_data.npy')
        temp_y = np.load(datapath+list_patient[LOOCV]+'_5min_data.npy')
        for idx in range(89):
            if idx == 0:
                idx_0 = 0
                idx_1 = 0
                idx_2 = 1
            if idx == 88:
                idx_0 = 87
                idx_1 = 88
                idx_2 = 88
            if idx > 0  and idx < 88:
                idx_0 = idx - 1
                idx_1 = idx
                idx_2 = idx + 1
            x_0 = temp_x[:,:,idx_0].reshape((192,192,1))
            x_1 = temp_x[:,:,idx_1].reshape((192,192,1))
            x_2 = temp_x[:,:,idx_2].reshape((192,192,1))
            x_stack = np.concatenate((x_0, x_1, x_2), axis=2)
            x_test[idx,:,:,:] = x_stack.reshape((1,192,192,slice_x)) / amp_x
            y_test[idx,:,:,:] = temp_y[:,:,idx].reshape((1,192,192,1)) / amp_y
        
        
        idx_x = 0
        idx_y = 0
        for patient in list_val:
            name_x = datapath+patient+'_1min_data.npy'
            name_y = datapath+patient+'_5min_data.npy'
            temp_x = np.load(name_x)
            temp_y = np.load(name_y)
            for idx in range(89):
                if idx == 0:
                    idx_0 = 0
                    idx_1 = 0
                    idx_2 = 1
                if idx == 88:
                    idx_0 = 87
                    idx_1 = 88
                    idx_2 = 88
                if idx > 0  and idx < 88:
                    idx_0 = idx - 1
                    idx_1 = idx
                    idx_2 = idx + 1
                x_0 = temp_x[:,:,idx_0].reshape((192,192,1))
                x_1 = temp_x[:,:,idx_1].reshape((192,192,1))
                x_2 = temp_x[:,:,idx_2].reshape((192,192,1))
                x_stack = np.concatenate((x_0, x_1, x_2), axis=2)
                x_val[idx_x,:,:,:] = x_stack.reshape((1,192,192,slice_x)) / amp_x
                y_val[idx_y,:,:,:] = temp_y[:,:,idx].reshape((1,192,192,1)) / amp_y
                idx_x = idx_x + 1
                idx_y = idx_y + 1 
        
        
        idx_x = 0
        idx_y = 0
        for patient in list_train:
            name_x = datapath+patient+'_1min_data.npy'
            name_y = datapath+patient+'_5min_data.npy'
            temp_x = np.load(name_x)
            temp_y = np.load(name_y)
            for idx in range(89):
                if idx == 0:
                    idx_0 = 0
                    idx_1 = 0
                    idx_2 = 1
                if idx == 88:
                    idx_0 = 87
                    idx_1 = 88
                    idx_2 = 88
                if idx > 0  and idx < 88:
                    idx_0 = idx - 1
                    idx_1 = idx
                    idx_2 = idx + 1
                x_0 = temp_x[:,:,idx_0].reshape((192,192,1))
                x_1 = temp_x[:,:,idx_1].reshape((192,192,1))
                x_2 = temp_x[:,:,idx_2].reshape((192,192,1))
                x_stack = np.concatenate((x_0, x_1, x_2), axis=2)
                x_train[idx_x,:,:,:] = x_stack.reshape((1,192,192,slice_x)) / amp_x
                y_train[idx_y,:,:,:] = temp_y[:,:,idx].reshape((1,192,192,1)) / amp_y
                idx_x = idx_x + 1
                idx_y = idx_y + 1     
    
    
    # PET-MR model
    if model_type == 'PET-MR':
        amp_x_PET = 14000
        amp_x_MR = 3000
        amp_y = 14000
        temp_x_PET = np.load(datapath+list_patient[LOOCV]+'_1min_data.npy')
        temp_x_MR = np.load(datapath+list_patient[LOOCV]+'_water_data.npy')
        temp_y = np.load(datapath+list_patient[LOOCV]+'_5min_data.npy')
        for idx in range(89):
            if idx == 0:
                idx_0 = 0
                idx_1 = 0
                idx_2 = 1
            if idx == 88:
                idx_0 = 87
                idx_1 = 88
                idx_2 = 88
            if idx > 0  and idx < 88:
                idx_0 = idx - 1
                idx_1 = idx
                idx_2 = idx + 1
            x_0 = temp_x_PET[:,:,idx_0].reshape((192,192,1)) / amp_x_PET
            x_1 = temp_x_PET[:,:,idx_1].reshape((192,192,1)) / amp_x_PET
            x_2 = temp_x_PET[:,:,idx_2].reshape((192,192,1)) / amp_x_PET
            x_3 = temp_x_MR[:,:,idx_1].reshape((192,192,1)) / amp_x_MR
            x_stack = np.concatenate((x_0, x_1, x_2, x_3), axis=2)
            x_test[idx,:,:,:] = x_stack.reshape((1,192,192,slice_x))
            y_test[idx,:,:,:] = temp_y[:,:,idx].reshape((1,192,192,1)) / amp_y
        
        
        idx_x = 0
        idx_y = 0
        for patient in list_val:
            name_x_PET = datapath+patient+'_1min_data.npy'
            name_x_MR = datapath+patient+'_water_data.npy'
            name_y = datapath+patient+'_5min_data.npy'
            temp_x_PET = np.load(name_x_PET)
            temp_x_MR = np.load(name_x_MR)
            temp_y = np.load(name_y)
            for idx in range(89):
                if idx == 0:
                    idx_0 = 0
                    idx_1 = 0
                    idx_2 = 1
                if idx == 88:
                    idx_0 = 87
                    idx_1 = 88
                    idx_2 = 88
                if idx > 0  and idx < 88:
                    idx_0 = idx - 1
                    idx_1 = idx
                    idx_2 = idx + 1
                x_0 = temp_x_PET[:,:,idx_0].reshape((192,192,1)) / amp_x_PET
                x_1 = temp_x_PET[:,:,idx_1].reshape((192,192,1)) / amp_x_PET
                x_2 = temp_x_PET[:,:,idx_2].reshape((192,192,1)) / amp_x_PET
                x_3 = temp_x_MR[:,:,idx_1].reshape((192,192,1)) / amp_x_MR
                x_stack = np.concatenate((x_0, x_1, x_2, x_3), axis=2)
                x_val[idx_x,:,:,:] = x_stack.reshape((1,192,192,slice_x))
                y_val[idx_y,:,:,:] = temp_y[:,:,idx].reshape((1,192,192,1)) / amp_y
                idx_x = idx_x + 1
                idx_y = idx_y + 1 
        
        
        idx_x = 0
        idx_y = 0
        for patient in list_train:
            name_x_PET = datapath+patient+'_1min_data.npy'
            name_x_MR = datapath+patient+'_water_data.npy'
            name_y = datapath+patient+'_5min_data.npy'
            temp_x_PET = np.load(name_x_PET)
            temp_x_MR = np.load(name_x_MR)
            temp_y = np.load(name_y)
            for idx in range(89):
                if idx == 0:
                    idx_0 = 0
                    idx_1 = 0
                    idx_2 = 1
                if idx == 88:
                    idx_0 = 87
                    idx_1 = 88
                    idx_2 = 88
                if idx > 0  and idx < 88:
                    idx_0 = idx - 1
                    idx_1 = idx
                    idx_2 = idx + 1
                x_0 = temp_x_PET[:,:,idx_0].reshape((192,192,1)) / amp_x_PET
                x_1 = temp_x_PET[:,:,idx_1].reshape((192,192,1)) / amp_x_PET
                x_2 = temp_x_PET[:,:,idx_2].reshape((192,192,1)) / amp_x_PET
                x_3 = temp_x_MR[:,:,idx_1].reshape((192,192,1)) / amp_x_MR
                x_stack = np.concatenate((x_0, x_1, x_2, x_3), axis=2)
                x_train[idx_x,:,:,:] = x_stack.reshape((1,192,192,slice_x))
                y_train[idx_y,:,:,:] = temp_y[:,:,idx].reshape((1,192,192,1)) / amp_y
                idx_x = idx_x + 1
                idx_y = idx_y + 1 
    
    
    #----------------------------------------------Data Generator----------------------------------------------#


    # train data_generator
    data_generator1 = ImageDataGenerator(rotation_range=conf_a["rotation_range"],
                                         shear_range=conf_a["shear_range"],
                                         width_shift_range=conf_a["width_shift_range"],
                                         height_shift_range=conf_a["height_shift_range"],
                                         zoom_range=conf_a["zoom_range"],
                                         horizontal_flip=conf_a["horizontal_flip"],
                                         vertical_flip=conf_a["vertical_flip"],
                                         fill_mode=conf_a["fill_mode"])
    data_generator2 = ImageDataGenerator(rotation_range=conf_a["rotation_range"],
                                         shear_range=conf_a["shear_range"],
                                         width_shift_range=conf_a["width_shift_range"],
                                         height_shift_range=conf_a["height_shift_range"],
                                         zoom_range=conf_a["zoom_range"],
                                         horizontal_flip=conf_a["horizontal_flip"],
                                         vertical_flip=conf_a["vertical_flip"],
                                         fill_mode=conf_a["fill_mode"])

    # validation data_generator
    data_generator3 = ImageDataGenerator(width_shift_range=conf_a["width_shift_range"],
                                         height_shift_range=conf_a["height_shift_range"],
                                         zoom_range=conf_a["zoom_range"],
                                         horizontal_flip=conf_a["horizontal_flip"],
                                         vertical_flip=conf_a["vertical_flip"],
                                         fill_mode=conf_a["fill_mode"])
    data_generator4 = ImageDataGenerator(width_shift_range=conf_a["width_shift_range"],
                                         height_shift_range=conf_a["height_shift_range"],
                                         zoom_range=conf_a["zoom_range"],
                                         horizontal_flip=conf_a["horizontal_flip"],
                                         vertical_flip=conf_a["vertical_flip"],
                                         fill_mode=conf_a["fill_mode"]) 


    # set generator
    data_generator1.fit(x_train, seed=conf_a["seed"])
    data_generator2.fit(y_train, seed=conf_a["seed"])
    data_generator3.fit(x_val, seed=conf_a["seed"])
    data_generator4.fit(y_val, seed=conf_a["seed"])
    data_generator_t = zip(data_generator1.flow(x=x_train, y=None, 
                        batch_size=conf_a["batch_size"], seed=conf_a["seed"]),
                        data_generator2.flow(x=y_train, y=None,
                        batch_size=conf_a["batch_size"], seed=conf_a["seed"]))
    data_generator_v = zip(data_generator3.flow(x=x_val, y=None, 
                        batch_size=conf_a["batch_size"], seed=conf_a["seed"]),
                        data_generator4.flow(x=y_val, y=None,
                        batch_size=conf_a["batch_size"], seed=conf_a["seed"]))


    #----------------------------------------------Train Model----------------------------------------------#


    # compile
    model.compile(loss=loss, optimizer=opt, metrics=[mean_squared_error_1e6, psnr])


    # train
    model.fit_generator(generator=data_generator_t, 
                        steps_per_epoch=int(89*14 / conf_a["batch_size"]), # 
                        epochs=conf["epochs"],
                        callbacks=callbacks_list,
                        validation_data=data_generator_v,
                        validation_steps=int(89*5 / conf_a["batch_size"])) # 
        
    
    #----------------------------------------------Test Model----------------------------------------------#
    
    del model
    gc.collect()
    model_path = '.\\training_models\\' + model_type + '_' + str(LOOCV) + '\\' + 'model*.hdf5'
    list_model = glob.glob(model_path)
    best_model = load_model(list_model[-1], 
                            custom_objects={'mean_squared_error_1e6': mean_squared_error_1e6,
                                            'psnr': psnr})
    print(list_model[-1])
    results = np.squeeze(best_model.predict(x_test))
    result_x = np.zeros((192,192,89))
    for idx in range(89):
        result_x[:,:,idx] = results[idx, :, :] * amp_y
    nii_path = ".\\dataset\\zero\\"
    syn_path = ".\\syn_PET\\"
    if not os.path.exists(syn_path):
        os.makedirs(syn_path)
    nii_file = nib.load(nii_path+list_patient[LOOCV]+'_1min.nii.gz')
    nii_file = nib.Nifti1Image(result_x, nii_file.affine, nii_file.header)
    nib.save(nii_file, syn_path+list_patient[LOOCV]+'_'+model_type+'_1min.nii.gz')
    print("Prediction!")
    
    
    #----------------------------------------------Release Memory----------------------------------------------#
    
    
    del x_train
    del y_train
    del x_val
    del y_val
    del x_test
    del y_test
    del result_x
    del best_model
    gc.collect()

In [9]:
depth = 4
n_fliter = 64
epochs = 400
loss = 'mse1e6'
optimizer = 'Adam'
batch_size = 5
model_type = 'PET-only' # 'PET-only', 'PET-MR', 'MR-only'
LOOCV = 7 # [0,19]

train_a_UNet(depth=depth, n_fliter=n_fliter,
             epochs=epochs, loss=loss,
             optimizer=optimizer, batch_size=batch_size,
             LOOCV=LOOCV, model_type=model_type)
print("Process is finished for "+model_type + '_' + str(LOOCV))

Epoch 1/400
249/249 [==============================] - 83s 334ms/step - loss: 11782.9001 - mean_squared_error_1e6: 11782.9001 - psnr: 44703.6986 - val_loss: 1985.8297 - val_mean_squared_error_1e6: 1985.8297 - val_psnr: 47504.0338

Epoch 00001: val_psnr improved from -inf to 47504.03384, saving model to .\training_models\PET-only_7\model.hdf5
Epoch 2/400
249/249 [==============================] - 79s 319ms/step - loss: 1797.0484 - mean_squared_error_1e6: 1797.0484 - psnr: 47834.4910 - val_loss: 1340.9118 - val_mean_squared_error_1e6: 1340.9118 - val_psnr: 49224.8994

Epoch 00002: val_psnr improved from 47504.03384 to 49224.89945, saving model to .\training_models\PET-only_7\model.hdf5
Epoch 3/400
249/249 [==============================] - 78s 315ms/step - loss: 1385.1388 - mean_squared_error_1e6: 1385.1388 - psnr: 48949.6852 - val_loss: 1115.9856 - val_mean_squared_error_1e6: 1115.9856 - val_psnr: 50065.8927

Epoch 00003: val_psnr improved from 49224.89945 to 50065.89273, saving model t


Epoch 00025: val_psnr improved from 59107.38452 to 59459.85305, saving model to .\training_models\PET-only_7\model.hdf5
Epoch 26/400
249/249 [==============================] - 77s 309ms/step - loss: 191.4938 - mean_squared_error_1e6: 191.4938 - psnr: 58238.7077 - val_loss: 180.4259 - val_mean_squared_error_1e6: 180.4259 - val_psnr: 58846.7276

Epoch 00026: val_psnr did not improve from 59459.85305
Epoch 27/400
249/249 [==============================] - 77s 309ms/step - loss: 183.1182 - mean_squared_error_1e6: 183.1182 - psnr: 58451.9109 - val_loss: 153.6691 - val_mean_squared_error_1e6: 153.6692 - val_psnr: 59633.4532

Epoch 00027: val_psnr improved from 59459.85305 to 59633.45321, saving model to .\training_models\PET-only_7\model.hdf5
Epoch 28/400
249/249 [==============================] - 77s 309ms/step - loss: 189.2733 - mean_squared_error_1e6: 189.2733 - psnr: 58342.2387 - val_loss: 158.6479 - val_mean_squared_error_1e6: 158.6480 - val_psnr: 59264.5710

Epoch 00028: val_psnr did 


Epoch 00081: val_psnr did not improve from 60107.35670
Epoch 82/400
249/249 [==============================] - 77s 308ms/step - loss: 147.5426 - mean_squared_error_1e6: 147.5427 - psnr: 59507.0975 - val_loss: 135.1052 - val_mean_squared_error_1e6: 135.1052 - val_psnr: 59750.4283

Epoch 00082: val_psnr did not improve from 60107.35670
Epoch 83/400
249/249 [==============================] - 77s 308ms/step - loss: 158.7476 - mean_squared_error_1e6: 158.7476 - psnr: 59095.2564 - val_loss: 136.6083 - val_mean_squared_error_1e6: 136.6084 - val_psnr: 59849.6455

Epoch 00083: val_psnr did not improve from 60107.35670
Epoch 84/400
249/249 [==============================] - 77s 308ms/step - loss: 145.4878 - mean_squared_error_1e6: 145.4879 - psnr: 59420.6906 - val_loss: 126.8067 - val_mean_squared_error_1e6: 126.8067 - val_psnr: 60038.2218

Epoch 00084: val_psnr did not improve from 60107.35670
Epoch 85/400
249/249 [==============================] - 77s 309ms/step - loss: 152.1261 - mean_square

249/249 [==============================] - 77s 308ms/step - loss: 154.4628 - mean_squared_error_1e6: 154.4628 - psnr: 59394.0513 - val_loss: 133.0759 - val_mean_squared_error_1e6: 133.0759 - val_psnr: 60015.9615

Epoch 00139: val_psnr did not improve from 60242.11517
Epoch 140/400
249/249 [==============================] - 77s 308ms/step - loss: 130.3165 - mean_squared_error_1e6: 130.3165 - psnr: 59954.5135 - val_loss: 135.0142 - val_mean_squared_error_1e6: 135.0143 - val_psnr: 60124.7333

Epoch 00140: val_psnr did not improve from 60242.11517
Epoch 141/400
249/249 [==============================] - 77s 308ms/step - loss: 165.3854 - mean_squared_error_1e6: 165.3855 - psnr: 59146.0740 - val_loss: 153.0999 - val_mean_squared_error_1e6: 153.1000 - val_psnr: 59988.7993

Epoch 00141: val_psnr did not improve from 60242.11517
Epoch 142/400
249/249 [==============================] - 77s 308ms/step - loss: 156.4401 - mean_squared_error_1e6: 156.4401 - psnr: 59332.4636 - val_loss: 145.5317 - va


Epoch 00196: val_psnr did not improve from 60377.76435
Epoch 197/400
249/249 [==============================] - 77s 307ms/step - loss: 164.9741 - mean_squared_error_1e6: 164.9741 - psnr: 59585.9573 - val_loss: 142.2484 - val_mean_squared_error_1e6: 142.2484 - val_psnr: 60022.9187

Epoch 00197: val_psnr did not improve from 60377.76435
Epoch 198/400
249/249 [==============================] - 77s 308ms/step - loss: 138.5675 - mean_squared_error_1e6: 138.5675 - psnr: 59718.0284 - val_loss: 128.2514 - val_mean_squared_error_1e6: 128.2514 - val_psnr: 60445.0229

Epoch 00198: val_psnr improved from 60377.76435 to 60445.02291, saving model to .\training_models\PET-only_7\model.hdf5
Epoch 199/400
249/249 [==============================] - 77s 308ms/step - loss: 131.7030 - mean_squared_error_1e6: 131.7030 - psnr: 59891.9579 - val_loss: 141.4935 - val_mean_squared_error_1e6: 141.4935 - val_psnr: 60053.6650

Epoch 00199: val_psnr did not improve from 60445.02291
Epoch 200/400
249/249 [==========


Epoch 00225: val_psnr did not improve from 60445.02291
Epoch 226/400
249/249 [==============================] - 77s 307ms/step - loss: 129.5423 - mean_squared_error_1e6: 129.5424 - psnr: 59946.8333 - val_loss: 135.5762 - val_mean_squared_error_1e6: 135.5762 - val_psnr: 59927.2069

Epoch 00226: val_psnr did not improve from 60445.02291
Epoch 227/400
249/249 [==============================] - 77s 308ms/step - loss: 128.1950 - mean_squared_error_1e6: 128.1950 - psnr: 59918.4543 - val_loss: 125.2658 - val_mean_squared_error_1e6: 125.2658 - val_psnr: 60109.1949

Epoch 00227: val_psnr did not improve from 60445.02291
Epoch 228/400
249/249 [==============================] - 77s 308ms/step - loss: 140.2400 - mean_squared_error_1e6: 140.2400 - psnr: 59685.8047 - val_loss: 136.7727 - val_mean_squared_error_1e6: 136.7727 - val_psnr: 59988.7544

Epoch 00228: val_psnr did not improve from 60445.02291
Epoch 229/400
249/249 [==============================] - 77s 308ms/step - loss: 129.8980 - mean_sq

249/249 [==============================] - 77s 307ms/step - loss: 133.7555 - mean_squared_error_1e6: 133.7555 - psnr: 59895.4649 - val_loss: 139.4198 - val_mean_squared_error_1e6: 139.4198 - val_psnr: 60035.7384

Epoch 00255: val_psnr did not improve from 60445.02291
Epoch 256/400
249/249 [==============================] - 77s 307ms/step - loss: 129.6023 - mean_squared_error_1e6: 129.6023 - psnr: 59970.5144 - val_loss: 128.4260 - val_mean_squared_error_1e6: 128.4260 - val_psnr: 60288.4310

Epoch 00256: val_psnr did not improve from 60445.02291
Epoch 257/400
249/249 [==============================] - 77s 307ms/step - loss: 133.6896 - mean_squared_error_1e6: 133.6896 - psnr: 59878.8178 - val_loss: 154.5412 - val_mean_squared_error_1e6: 154.5412 - val_psnr: 60140.0564

Epoch 00257: val_psnr did not improve from 60445.02291
Epoch 258/400
249/249 [==============================] - 77s 308ms/step - loss: 131.9917 - mean_squared_error_1e6: 131.9917 - psnr: 59888.6335 - val_loss: 129.7557 - va

249/249 [==============================] - 77s 307ms/step - loss: 139.7916 - mean_squared_error_1e6: 139.7916 - psnr: 59819.0539 - val_loss: 213.2221 - val_mean_squared_error_1e6: 213.2221 - val_psnr: 60301.1595

Epoch 00313: val_psnr did not improve from 60500.24592
Epoch 314/400
249/249 [==============================] - 77s 308ms/step - loss: 116.2128 - mean_squared_error_1e6: 116.2128 - psnr: 60231.8923 - val_loss: 131.2785 - val_mean_squared_error_1e6: 131.2785 - val_psnr: 60310.9019

Epoch 00314: val_psnr did not improve from 60500.24592
Epoch 315/400
249/249 [==============================] - 77s 308ms/step - loss: 127.7381 - mean_squared_error_1e6: 127.7381 - psnr: 60106.5604 - val_loss: 181.7093 - val_mean_squared_error_1e6: 181.7093 - val_psnr: 59783.1467

Epoch 00315: val_psnr did not improve from 60500.24592
Epoch 316/400
249/249 [==============================] - 76s 307ms/step - loss: 135.8557 - mean_squared_error_1e6: 135.8557 - psnr: 59795.7142 - val_loss: 130.7067 - va

249/249 [==============================] - 77s 308ms/step - loss: 125.3726 - mean_squared_error_1e6: 125.3726 - psnr: 60219.1075 - val_loss: 140.1353 - val_mean_squared_error_1e6: 140.1353 - val_psnr: 60260.9389

Epoch 00371: val_psnr did not improve from 60500.24592
Epoch 372/400
249/249 [==============================] - 77s 308ms/step - loss: 127.9197 - mean_squared_error_1e6: 127.9198 - psnr: 60131.2632 - val_loss: 123.0388 - val_mean_squared_error_1e6: 123.0388 - val_psnr: 60510.9699

Epoch 00372: val_psnr improved from 60500.24592 to 60510.96994, saving model to .\training_models\PET-only_7\model.hdf5
Epoch 373/400
249/249 [==============================] - 77s 308ms/step - loss: 114.8376 - mean_squared_error_1e6: 114.8376 - psnr: 60402.0325 - val_loss: 129.4458 - val_mean_squared_error_1e6: 129.4459 - val_psnr: 60209.9472

Epoch 00373: val_psnr did not improve from 60510.96994
Epoch 374/400
249/249 [==============================] - 77s 308ms/step - loss: 119.2440 - mean_squared

Epoch 400/400
249/249 [==============================] - 76s 307ms/step - loss: 121.8709 - mean_squared_error_1e6: 121.8709 - psnr: 60066.8499 - val_loss: 129.3159 - val_mean_squared_error_1e6: 129.3159 - val_psnr: 60201.1788

Epoch 00400: val_psnr did not improve from 60510.96994
.\training_models\PET-only_7\model.hdf5
Prediction!
Process is finished for PET-only_7


In [ ]:
depth = 4
n_fliter = 64
epochs = 400
loss = 'mse1e6'
optimizer = 'Adam'
batch_size = 5
model_type = 'PET-MR' # 'PET-only', 'PET-MR', 'MR-only'
LOOCV = 10 # [0,19]

train_a_UNet(depth=depth, n_fliter=n_fliter,
             epochs=epochs, loss=loss,
             optimizer=optimizer, batch_size=batch_size,
             LOOCV=LOOCV, model_type=model_type)
print("Process is finished for "+model_type + '_' + str(LOOCV))

Epoch 1/400
249/249 [==============================] - 81s 324ms/step - loss: 11402.8441 - mean_squared_error_1e6: 11402.8439 - psnr: 44598.4790 - val_loss: 1823.4787 - val_mean_squared_error_1e6: 1823.4787 - val_psnr: 47882.9866

Epoch 00001: val_psnr improved from -inf to 47882.98661, saving model to .\training_models\PET-MR_10\model.hdf5
Epoch 2/400
249/249 [==============================] - 77s 310ms/step - loss: 2126.9148 - mean_squared_error_1e6: 2126.9148 - psnr: 47936.9084 - val_loss: 1102.4029 - val_mean_squared_error_1e6: 1102.4029 - val_psnr: 49990.9114

Epoch 00002: val_psnr improved from 47882.98661 to 49990.91143, saving model to .\training_models\PET-MR_10\model.hdf5
Epoch 3/400
249/249 [==============================] - 78s 312ms/step - loss: 1690.9635 - mean_squared_error_1e6: 1690.9635 - psnr: 49039.0784 - val_loss: 895.3562 - val_mean_squared_error_1e6: 895.3562 - val_psnr: 50921.6647

Epoch 00003: val_psnr improved from 49990.91143 to 50921.66468, saving model to .\

In [9]:
depth = 4
n_fliter = 64
epochs = 400
loss = 'mse1e6'
optimizer = 'Adam'
batch_size = 5
model_type = 'MR-only' # 'PET-only', 'PET-MR', 'MR-only'
LOOCV = 13 # [0,19]

train_a_UNet(depth=depth, n_fliter=n_fliter,
             epochs=epochs, loss=loss,
             optimizer=optimizer, batch_size=batch_size,
             LOOCV=LOOCV, model_type=model_type)
print("Process is finished for "+model_type + '_' + str(LOOCV))

Epoch 1/400
249/249 [==============================] - 83s 332ms/step - loss: 11686.9914 - mean_squared_error_1e6: 11686.9915 - psnr: 45115.9809 - val_loss: 1468.8526 - val_mean_squared_error_1e6: 1468.8526 - val_psnr: 48702.9476

Epoch 00001: val_psnr improved from -inf to 48702.94764, saving model to .\training_models\MR-only_13\model.hdf5
Epoch 2/400
249/249 [==============================] - 79s 318ms/step - loss: 2482.5699 - mean_squared_error_1e6: 2482.5699 - psnr: 47744.7860 - val_loss: 1303.6055 - val_mean_squared_error_1e6: 1303.6055 - val_psnr: 49262.5245

Epoch 00002: val_psnr improved from 48702.94764 to 49262.52449, saving model to .\training_models\MR-only_13\model.hdf5
Epoch 3/400
249/249 [==============================] - 78s 314ms/step - loss: 2324.0234 - mean_squared_error_1e6: 2324.0234 - psnr: 48140.1194 - val_loss: 1172.6131 - val_mean_squared_error_1e6: 1172.6131 - val_psnr: 49784.0796

Epoch 00003: val_psnr improved from 49262.52449 to 49784.07962, saving model t

Epoch 55/400
249/249 [==============================] - 77s 308ms/step - loss: 1000.1744 - mean_squared_error_1e6: 1000.1745 - psnr: 52405.1757 - val_loss: 609.3915 - val_mean_squared_error_1e6: 609.3915 - val_psnr: 52762.9876

Epoch 00055: val_psnr did not improve from 52956.96721
Epoch 56/400
249/249 [==============================] - 77s 307ms/step - loss: 1120.5605 - mean_squared_error_1e6: 1120.5607 - psnr: 52053.9755 - val_loss: 761.5840 - val_mean_squared_error_1e6: 761.5841 - val_psnr: 51944.8486

Epoch 00056: val_psnr did not improve from 52956.96721
Epoch 57/400
249/249 [==============================] - 77s 308ms/step - loss: 1031.0909 - mean_squared_error_1e6: 1031.0911 - psnr: 52074.8699 - val_loss: 695.2031 - val_mean_squared_error_1e6: 695.2032 - val_psnr: 52180.0893

Epoch 00057: val_psnr did not improve from 52956.96721
Epoch 58/400
249/249 [==============================] - 77s 308ms/step - loss: 910.9938 - mean_squared_error_1e6: 910.9939 - psnr: 52480.6287 - val_los

249/249 [==============================] - 76s 307ms/step - loss: 743.5077 - mean_squared_error_1e6: 743.5079 - psnr: 53805.7283 - val_loss: 568.0216 - val_mean_squared_error_1e6: 568.0218 - val_psnr: 53239.7302

Epoch 00111: val_psnr did not improve from 53480.29450
Epoch 112/400
249/249 [==============================] - 76s 307ms/step - loss: 841.4775 - mean_squared_error_1e6: 841.4777 - psnr: 53530.4226 - val_loss: 541.8680 - val_mean_squared_error_1e6: 541.8681 - val_psnr: 53241.5729

Epoch 00112: val_psnr did not improve from 53480.29450
Epoch 113/400
249/249 [==============================] - 76s 307ms/step - loss: 875.3717 - mean_squared_error_1e6: 875.3719 - psnr: 53448.7154 - val_loss: 675.9058 - val_mean_squared_error_1e6: 675.9059 - val_psnr: 52706.8116

Epoch 00113: val_psnr did not improve from 53480.29450
Epoch 114/400
249/249 [==============================] - 76s 306ms/step - loss: 762.9639 - mean_squared_error_1e6: 762.9640 - psnr: 53693.4766 - val_loss: 548.3065 - va


Epoch 00139: val_psnr did not improve from 53717.21072
Epoch 140/400
249/249 [==============================] - 76s 306ms/step - loss: 737.7820 - mean_squared_error_1e6: 737.7821 - psnr: 54163.5355 - val_loss: 685.6095 - val_mean_squared_error_1e6: 685.6096 - val_psnr: 52757.1217

Epoch 00140: val_psnr did not improve from 53717.21072
Epoch 141/400
249/249 [==============================] - 76s 306ms/step - loss: 828.0539 - mean_squared_error_1e6: 828.0540 - psnr: 53799.5032 - val_loss: 530.5381 - val_mean_squared_error_1e6: 530.5382 - val_psnr: 53347.8235

Epoch 00141: val_psnr did not improve from 53717.21072
Epoch 142/400
249/249 [==============================] - 76s 306ms/step - loss: 867.6924 - mean_squared_error_1e6: 867.6925 - psnr: 53624.6820 - val_loss: 623.7153 - val_mean_squared_error_1e6: 623.7154 - val_psnr: 52821.2121

Epoch 00142: val_psnr did not improve from 53717.21072
Epoch 143/400
249/249 [==============================] - 76s 306ms/step - loss: 794.1752 - mean_sq


Epoch 00168: val_psnr did not improve from 53730.91310
Epoch 169/400
249/249 [==============================] - 76s 307ms/step - loss: 725.3252 - mean_squared_error_1e6: 725.3253 - psnr: 54173.3130 - val_loss: 645.8583 - val_mean_squared_error_1e6: 645.8585 - val_psnr: 52362.3899

Epoch 00169: val_psnr did not improve from 53730.91310
Epoch 170/400
249/249 [==============================] - 77s 307ms/step - loss: 768.6484 - mean_squared_error_1e6: 768.6486 - psnr: 54366.9700 - val_loss: 558.3745 - val_mean_squared_error_1e6: 558.3746 - val_psnr: 53290.1270

Epoch 00170: val_psnr did not improve from 53730.91310
Epoch 171/400
249/249 [==============================] - 76s 307ms/step - loss: 859.3361 - mean_squared_error_1e6: 859.3362 - psnr: 54158.9877 - val_loss: 542.8577 - val_mean_squared_error_1e6: 542.8578 - val_psnr: 53464.9313

Epoch 00171: val_psnr did not improve from 53730.91310
Epoch 172/400
249/249 [==============================] - 76s 306ms/step - loss: 763.8557 - mean_sq

...

249/249 [==============================] - 76s 307ms/step - loss: 749.7559 - mean_squared_error_1e6: 749.7560 - psnr: 55086.3479 - val_loss: 520.0632 - val_mean_squared_error_1e6: 520.0633 - val_psnr: 53677.5039

Epoch 00256: val_psnr did not improve from 53894.46656
Epoch 257/400
249/249 [==============================] - 76s 306ms/step - loss: 724.9332 - mean_squared_error_1e6: 724.9333 - psnr: 55147.5515 - val_loss: 502.9582 - val_mean_squared_error_1e6: 502.9583 - val_psnr: 53872.9662

Epoch 00257: val_psnr did not improve from 53894.46656
Epoch 258/400
249/249 [==============================] - 76s 306ms/step - loss: 721.7907 - mean_squared_error_1e6: 721.7908 - psnr: 54903.3884 - val_loss: 516.3366 - val_mean_squared_error_1e6: 516.3367 - val_psnr: 53737.4153

Epoch 00258: val_psnr did not improve from 53894.46656
Epoch 259/400
249/249 [==============================] - 76s 307ms/step - loss: 714.6035 - mean_squared_error_1e6: 714.6036 - psnr: 55240.0934 - val_loss: 480.1715 - va

249/249 [==============================] - 76s 306ms/step - loss: 759.6955 - mean_squared_error_1e6: 759.6956 - psnr: 55456.6387 - val_loss: 530.8740 - val_mean_squared_error_1e6: 530.8741 - val_psnr: 53652.8351

Epoch 00285: val_psnr did not improve from 53900.39221
Epoch 286/400
249/249 [==============================] - 76s 306ms/step - loss: 671.9813 - mean_squared_error_1e6: 671.9814 - psnr: 55460.1298 - val_loss: 520.0567 - val_mean_squared_error_1e6: 520.0569 - val_psnr: 53692.0337

Epoch 00286: val_psnr did not improve from 53900.39221
Epoch 287/400
249/249 [==============================] - 76s 306ms/step - loss: 714.2894 - mean_squared_error_1e6: 714.2895 - psnr: 55263.0114 - val_loss: 557.0111 - val_mean_squared_error_1e6: 557.0112 - val_psnr: 53570.0545

Epoch 00287: val_psnr did not improve from 53900.39221
Epoch 288/400
249/249 [==============================] - 76s 306ms/step - loss: 719.4433 - mean_squared_error_1e6: 719.4434 - psnr: 55369.0671 - val_loss: 499.0478 - va

249/249 [==============================] - 76s 307ms/step - loss: 768.4607 - mean_squared_error_1e6: 768.4608 - psnr: 55617.3114 - val_loss: 488.2592 - val_mean_squared_error_1e6: 488.2593 - val_psnr: 54084.7445

Epoch 00342: val_psnr did not improve from 54126.52932
Epoch 343/400
249/249 [==============================] - 76s 306ms/step - loss: 665.7421 - mean_squared_error_1e6: 665.7422 - psnr: 55793.7399 - val_loss: 504.0182 - val_mean_squared_error_1e6: 504.0183 - val_psnr: 53949.5067

Epoch 00343: val_psnr did not improve from 54126.52932
Epoch 344/400
249/249 [==============================] - 76s 307ms/step - loss: 641.5412 - mean_squared_error_1e6: 641.5413 - psnr: 55849.8349 - val_loss: 486.6418 - val_mean_squared_error_1e6: 486.6419 - val_psnr: 53899.9839

Epoch 00344: val_psnr did not improve from 54126.52932
Epoch 345/400
249/249 [==============================] - 76s 307ms/step - loss: 613.7907 - mean_squared_error_1e6: 613.7908 - psnr: 55942.6316 - val_loss: 528.6431 - va

Epoch 371/400
249/249 [==============================] - 76s 307ms/step - loss: 773.7765 - mean_squared_error_1e6: 773.7766 - psnr: 55682.5839 - val_loss: 501.4997 - val_mean_squared_error_1e6: 501.4998 - val_psnr: 53979.1117

Epoch 00371: val_psnr did not improve from 54208.71335
Epoch 372/400
249/249 [==============================] - 76s 306ms/step - loss: 661.5006 - mean_squared_error_1e6: 661.5007 - psnr: 55897.3820 - val_loss: 476.0236 - val_mean_squared_error_1e6: 476.0237 - val_psnr: 54064.4639

Epoch 00372: val_psnr did not improve from 54208.71335
Epoch 373/400
249/249 [==============================] - 76s 306ms/step - loss: 687.1953 - mean_squared_error_1e6: 687.1953 - psnr: 55883.3895 - val_loss: 516.4482 - val_mean_squared_error_1e6: 516.4483 - val_psnr: 53882.2306

Epoch 00373: val_psnr did not improve from 54208.71335
Epoch 374/400
249/249 [==============================] - 76s 306ms/step - loss: 557.6111 - mean_squared_error_1e6: 557.6112 - psnr: 56250.2015 - val_loss:

Epoch 400/400
249/249 [==============================] - 76s 307ms/step - loss: 612.5060 - mean_squared_error_1e6: 612.5060 - psnr: 55906.2094 - val_loss: 494.1994 - val_mean_squared_error_1e6: 494.1995 - val_psnr: 53959.3050

Epoch 00400: val_psnr did not improve from 54329.33804
.\training_models\MR-only_13\model.hdf5
Prediction!
Process is finished for MR-only_13


In [9]:
depth = 4
n_fliter = 64
epochs = 400
loss = 'mse1e6'
optimizer = 'Adam'
batch_size = 5
model_type = 'MR-only' # 'PET-only', 'PET-MR', 'MR-only'
LOOCV = 11 # [0,19]

train_a_UNet(depth=depth, n_fliter=n_fliter,
             epochs=epochs, loss=loss,
             optimizer=optimizer, batch_size=batch_size,
             LOOCV=LOOCV, model_type=model_type)
print("Process is finished for "+model_type + '_' + str(LOOCV))

Epoch 1/400
249/249 [==============================] - 82s 330ms/step - loss: 11748.7837 - mean_squared_error_1e6: 11748.7835 - psnr: 45084.8765 - val_loss: 1532.3318 - val_mean_squared_error_1e6: 1532.3318 - val_psnr: 48598.6952

Epoch 00001: val_psnr improved from -inf to 48598.69522, saving model to .\training_models\MR-only_11\model.hdf5
Epoch 2/400
249/249 [==============================] - 78s 313ms/step - loss: 2412.9189 - mean_squared_error_1e6: 2412.9189 - psnr: 47785.2357 - val_loss: 1330.8337 - val_mean_squared_error_1e6: 1330.8337 - val_psnr: 49207.0169

Epoch 00002: val_psnr improved from 48598.69522 to 49207.01694, saving model to .\training_models\MR-only_11\model.hdf5
Epoch 3/400
249/249 [==============================] - 77s 307ms/step - loss: 2221.2499 - mean_squared_error_1e6: 2221.2499 - psnr: 48212.3645 - val_loss: 1306.8479 - val_mean_squared_error_1e6: 1306.8479 - val_psnr: 49323.8725

Epoch 00003: val_psnr improved from 49207.01694 to 49323.87250, saving model t

Epoch 27/400
249/249 [==============================] - 78s 312ms/step - loss: 1118.5772 - mean_squared_error_1e6: 1118.5774 - psnr: 51415.0884 - val_loss: 732.0770 - val_mean_squared_error_1e6: 732.0771 - val_psnr: 51791.6696

Epoch 00027: val_psnr did not improve from 52480.97858
Epoch 28/400
249/249 [==============================] - 77s 309ms/step - loss: 1139.3182 - mean_squared_error_1e6: 1139.3184 - psnr: 51351.7741 - val_loss: 981.2501 - val_mean_squared_error_1e6: 981.2502 - val_psnr: 50369.5778

Epoch 00028: val_psnr did not improve from 52480.97858
Epoch 29/400
249/249 [==============================] - 77s 310ms/step - loss: 1231.1926 - mean_squared_error_1e6: 1231.1929 - psnr: 51127.1354 - val_loss: 795.2898 - val_mean_squared_error_1e6: 795.2899 - val_psnr: 51663.5457

Epoch 00029: val_psnr did not improve from 52480.97858
Epoch 30/400
249/249 [==============================] - 77s 309ms/step - loss: 1249.6749 - mean_squared_error_1e6: 1249.6751 - psnr: 51135.4917 - val_l


Epoch 00055: val_psnr did not improve from 52954.69996
Epoch 56/400
249/249 [==============================] - 77s 308ms/step - loss: 1112.0319 - mean_squared_error_1e6: 1112.0321 - psnr: 51961.7623 - val_loss: 805.4630 - val_mean_squared_error_1e6: 805.4631 - val_psnr: 51874.5123

Epoch 00056: val_psnr did not improve from 52954.69996
Epoch 57/400
249/249 [==============================] - 77s 308ms/step - loss: 1054.7117 - mean_squared_error_1e6: 1054.7119 - psnr: 51922.1294 - val_loss: 1115.3757 - val_mean_squared_error_1e6: 1115.3759 - val_psnr: 50619.0672

Epoch 00057: val_psnr did not improve from 52954.69996
Epoch 58/400
249/249 [==============================] - 77s 307ms/step - loss: 940.7172 - mean_squared_error_1e6: 940.7174 - psnr: 52344.2488 - val_loss: 728.6684 - val_mean_squared_error_1e6: 728.6686 - val_psnr: 52184.5237

Epoch 00058: val_psnr did not improve from 52954.69996
Epoch 59/400
249/249 [==============================] - 77s 308ms/step - loss: 1007.9631 - mean

Epoch 85/400
249/249 [==============================] - 76s 307ms/step - loss: 1119.1201 - mean_squared_error_1e6: 1119.1203 - psnr: 52733.5301 - val_loss: 897.9879 - val_mean_squared_error_1e6: 897.9881 - val_psnr: 51669.4275

Epoch 00085: val_psnr did not improve from 52954.69996
Epoch 86/400
249/249 [==============================] - 77s 307ms/step - loss: 950.8564 - mean_squared_error_1e6: 950.8565 - psnr: 53051.7521 - val_loss: 668.9147 - val_mean_squared_error_1e6: 668.9148 - val_psnr: 52349.5033

Epoch 00086: val_psnr did not improve from 52954.69996
Epoch 87/400
249/249 [==============================] - 77s 307ms/step - loss: 933.6129 - mean_squared_error_1e6: 933.6130 - psnr: 52685.3731 - val_loss: 610.2699 - val_mean_squared_error_1e6: 610.2700 - val_psnr: 52632.5292

Epoch 00087: val_psnr did not improve from 52954.69996
Epoch 88/400
249/249 [==============================] - 76s 307ms/step - loss: 820.3154 - mean_squared_error_1e6: 820.3156 - psnr: 53372.6507 - val_loss: 6


Epoch 00113: val_psnr did not improve from 53182.87144
Epoch 114/400
249/249 [==============================] - 76s 307ms/step - loss: 758.6347 - mean_squared_error_1e6: 758.6348 - psnr: 53716.0601 - val_loss: 639.9625 - val_mean_squared_error_1e6: 639.9626 - val_psnr: 52823.2967

Epoch 00114: val_psnr did not improve from 53182.87144
Epoch 115/400
249/249 [==============================] - 76s 307ms/step - loss: 845.2922 - mean_squared_error_1e6: 845.2923 - psnr: 53651.7720 - val_loss: 635.3839 - val_mean_squared_error_1e6: 635.3840 - val_psnr: 52805.7019

Epoch 00115: val_psnr did not improve from 53182.87144
Epoch 116/400
249/249 [==============================] - 76s 307ms/step - loss: 874.9798 - mean_squared_error_1e6: 874.9799 - psnr: 53688.9643 - val_loss: 588.3591 - val_mean_squared_error_1e6: 588.3592 - val_psnr: 52998.9282

Epoch 00116: val_psnr did not improve from 53182.87144
Epoch 117/400
249/249 [==============================] - 76s 307ms/step - loss: 734.0336 - mean_sq

...


Epoch 00199: val_psnr did not improve from 53659.12768
Epoch 200/400
249/249 [==============================] - 76s 307ms/step - loss: 910.6393 - mean_squared_error_1e6: 910.6395 - psnr: 54482.9003 - val_loss: 571.4049 - val_mean_squared_error_1e6: 571.4050 - val_psnr: 53300.6803

Epoch 00200: val_psnr did not improve from 53659.12768
Epoch 201/400
249/249 [==============================] - 77s 307ms/step - loss: 660.0914 - mean_squared_error_1e6: 660.0915 - psnr: 54869.9394 - val_loss: 582.1141 - val_mean_squared_error_1e6: 582.1142 - val_psnr: 53264.4013

Epoch 00201: val_psnr did not improve from 53659.12768
Epoch 202/400
249/249 [==============================] - 76s 307ms/step - loss: 874.4152 - mean_squared_error_1e6: 874.4153 - psnr: 54566.8629 - val_loss: 571.2512 - val_mean_squared_error_1e6: 571.2513 - val_psnr: 53348.8948

Epoch 00202: val_psnr did not improve from 53659.12768
Epoch 203/400
249/249 [==============================] - 76s 307ms/step - loss: 733.0694 - mean_sq

Epoch 229/400
249/249 [==============================] - 78s 314ms/step - loss: 775.2854 - mean_squared_error_1e6: 775.2855 - psnr: 54977.2761 - val_loss: 600.3738 - val_mean_squared_error_1e6: 600.3739 - val_psnr: 53158.7143

Epoch 00229: val_psnr did not improve from 53659.12768
Epoch 230/400
249/249 [==============================] - 77s 308ms/step - loss: 784.8532 - mean_squared_error_1e6: 784.8533 - psnr: 54932.2076 - val_loss: 550.5936 - val_mean_squared_error_1e6: 550.5937 - val_psnr: 53442.3052

Epoch 00230: val_psnr did not improve from 53659.12768
Epoch 231/400
249/249 [==============================] - 77s 308ms/step - loss: 630.9068 - mean_squared_error_1e6: 630.9069 - psnr: 55232.5170 - val_loss: 607.7565 - val_mean_squared_error_1e6: 607.7565 - val_psnr: 52961.9423

Epoch 00231: val_psnr did not improve from 53659.12768
Epoch 232/400
249/249 [==============================] - 77s 307ms/step - loss: 809.3695 - mean_squared_error_1e6: 809.3696 - psnr: 54835.1652 - val_loss:

249/249 [==============================] - 77s 307ms/step - loss: 731.3020 - mean_squared_error_1e6: 731.3021 - psnr: 54819.8721 - val_loss: 586.5057 - val_mean_squared_error_1e6: 586.5058 - val_psnr: 53291.3397

Epoch 00258: val_psnr did not improve from 53840.76931
Epoch 259/400
249/249 [==============================] - 77s 308ms/step - loss: 719.0206 - mean_squared_error_1e6: 719.0207 - psnr: 55168.2266 - val_loss: 547.3490 - val_mean_squared_error_1e6: 547.3491 - val_psnr: 53382.6691

Epoch 00259: val_psnr did not improve from 53840.76931
Epoch 260/400
249/249 [==============================] - 77s 310ms/step - loss: 691.5024 - mean_squared_error_1e6: 691.5025 - psnr: 55224.1626 - val_loss: 506.0927 - val_mean_squared_error_1e6: 506.0928 - val_psnr: 53679.3337

Epoch 00260: val_psnr did not improve from 53840.76931
Epoch 261/400
249/249 [==============================] - 77s 310ms/step - loss: 671.6195 - mean_squared_error_1e6: 671.6195 - psnr: 55380.3870 - val_loss: 569.6883 - va

Epoch 287/400
249/249 [==============================] - 78s 315ms/step - loss: 717.6185 - mean_squared_error_1e6: 717.6186 - psnr: 55195.3481 - val_loss: 555.3447 - val_mean_squared_error_1e6: 555.3448 - val_psnr: 53417.8255

Epoch 00287: val_psnr did not improve from 53978.32597
Epoch 288/400
249/249 [==============================] - 78s 315ms/step - loss: 720.2841 - mean_squared_error_1e6: 720.2842 - psnr: 55373.8762 - val_loss: 538.7455 - val_mean_squared_error_1e6: 538.7456 - val_psnr: 53446.8519

Epoch 00288: val_psnr did not improve from 53978.32597
Epoch 289/400
249/249 [==============================] - 78s 314ms/step - loss: 637.6962 - mean_squared_error_1e6: 637.6963 - psnr: 55426.7860 - val_loss: 567.1892 - val_mean_squared_error_1e6: 567.1893 - val_psnr: 53234.3603

Epoch 00289: val_psnr did not improve from 53978.32597
Epoch 290/400
249/249 [==============================] - 78s 314ms/step - loss: 730.1347 - mean_squared_error_1e6: 730.1348 - psnr: 55285.6968 - val_loss:

249/249 [==============================] - 78s 314ms/step - loss: 621.5904 - mean_squared_error_1e6: 621.5905 - psnr: 55796.8205 - val_loss: 542.8377 - val_mean_squared_error_1e6: 542.8378 - val_psnr: 53535.8721

Epoch 00345: val_psnr did not improve from 53978.32597
Epoch 346/400
249/249 [==============================] - 78s 314ms/step - loss: 752.3842 - mean_squared_error_1e6: 752.3842 - psnr: 55617.0399 - val_loss: 522.7849 - val_mean_squared_error_1e6: 522.7850 - val_psnr: 53756.3872

Epoch 00346: val_psnr did not improve from 53978.32597
Epoch 347/400
249/249 [==============================] - 78s 314ms/step - loss: 700.8388 - mean_squared_error_1e6: 700.8389 - psnr: 55973.7284 - val_loss: 509.6558 - val_mean_squared_error_1e6: 509.6559 - val_psnr: 53859.0580

Epoch 00347: val_psnr did not improve from 53978.32597
Epoch 348/400
249/249 [==============================] - 78s 314ms/step - loss: 685.3267 - mean_squared_error_1e6: 685.3268 - psnr: 55443.6864 - val_loss: 521.3392 - va

In [9]:
depth = 4
n_fliter = 64
epochs = 400
loss = 'mse1e6'
optimizer = 'Adam'
batch_size = 5
model_type = 'MR-only' # 'PET-only', 'PET-MR', 'MR-only'
LOOCV = 10 # [0,19]

train_a_UNet(depth=depth, n_fliter=n_fliter,
             epochs=epochs, loss=loss,
             optimizer=optimizer, batch_size=batch_size,
             LOOCV=LOOCV, model_type=model_type)
print("Process is finished for "+model_type + '_' + str(LOOCV))

Epoch 1/400
249/249 [==============================] - 82s 329ms/step - loss: 11619.9150 - mean_squared_error_1e6: 11619.9151 - psnr: 45187.3538 - val_loss: 1473.1102 - val_mean_squared_error_1e6: 1473.1102 - val_psnr: 48678.3076

Epoch 00001: val_psnr improved from -inf to 48678.30763, saving model to .\training_models\MR-only_10\model.hdf5
Epoch 2/400
249/249 [==============================] - 79s 318ms/step - loss: 2432.5961 - mean_squared_error_1e6: 2432.5961 - psnr: 47868.6895 - val_loss: 1242.5447 - val_mean_squared_error_1e6: 1242.5447 - val_psnr: 49411.1028

Epoch 00002: val_psnr improved from 48678.30763 to 49411.10284, saving model to .\training_models\MR-only_10\model.hdf5
Epoch 3/400
249/249 [==============================] - 78s 313ms/step - loss: 2297.8192 - mean_squared_error_1e6: 2297.8192 - psnr: 48203.9684 - val_loss: 1215.5612 - val_mean_squared_error_1e6: 1215.5612 - val_psnr: 49621.9510

Epoch 00003: val_psnr improved from 49411.10284 to 49621.95102, saving model t


Epoch 00027: val_psnr did not improve from 52439.56706
Epoch 28/400
249/249 [==============================] - 78s 314ms/step - loss: 1137.5848 - mean_squared_error_1e6: 1137.5849 - psnr: 51442.0142 - val_loss: 741.6013 - val_mean_squared_error_1e6: 741.6013 - val_psnr: 51605.8161

Epoch 00028: val_psnr did not improve from 52439.56706
Epoch 29/400
249/249 [==============================] - 78s 313ms/step - loss: 1269.8501 - mean_squared_error_1e6: 1269.8503 - psnr: 51096.5175 - val_loss: 914.5971 - val_mean_squared_error_1e6: 914.5972 - val_psnr: 51699.9414

Epoch 00029: val_psnr did not improve from 52439.56706
Epoch 30/400
249/249 [==============================] - 78s 313ms/step - loss: 1239.9843 - mean_squared_error_1e6: 1239.9845 - psnr: 51237.5001 - val_loss: 1145.9270 - val_mean_squared_error_1e6: 1145.9271 - val_psnr: 50606.8118

Epoch 00030: val_psnr did not improve from 52439.56706
Epoch 31/400
249/249 [==============================] - 78s 314ms/step - loss: 1097.1719 - me

Epoch 84/400
249/249 [==============================] - 76s 306ms/step - loss: 743.1873 - mean_squared_error_1e6: 743.1874 - psnr: 53609.2904 - val_loss: 650.4321 - val_mean_squared_error_1e6: 650.4322 - val_psnr: 52441.8386

Epoch 00084: val_psnr did not improve from 53210.91924
Epoch 85/400
249/249 [==============================] - 76s 306ms/step - loss: 993.5972 - mean_squared_error_1e6: 993.5973 - psnr: 53026.0707 - val_loss: 604.9616 - val_mean_squared_error_1e6: 604.9617 - val_psnr: 53035.4084

Epoch 00085: val_psnr did not improve from 53210.91924
Epoch 86/400
249/249 [==============================] - 76s 305ms/step - loss: 931.4463 - mean_squared_error_1e6: 931.4465 - psnr: 53211.7973 - val_loss: 594.6747 - val_mean_squared_error_1e6: 594.6748 - val_psnr: 52780.3509

Epoch 00086: val_psnr did not improve from 53210.91924
Epoch 87/400
249/249 [==============================] - 76s 306ms/step - loss: 872.8865 - mean_squared_error_1e6: 872.8866 - psnr: 53105.4230 - val_loss: 574


Epoch 00112: val_psnr did not improve from 53378.00342
Epoch 113/400
249/249 [==============================] - 76s 306ms/step - loss: 846.0327 - mean_squared_error_1e6: 846.0329 - psnr: 53746.5259 - val_loss: 667.7864 - val_mean_squared_error_1e6: 667.7864 - val_psnr: 52758.9790

Epoch 00113: val_psnr did not improve from 53378.00342
Epoch 114/400
249/249 [==============================] - 76s 306ms/step - loss: 731.6823 - mean_squared_error_1e6: 731.6824 - psnr: 53981.5508 - val_loss: 593.8590 - val_mean_squared_error_1e6: 593.8591 - val_psnr: 53046.4878

Epoch 00114: val_psnr did not improve from 53378.00342
Epoch 115/400
249/249 [==============================] - 76s 306ms/step - loss: 820.7968 - mean_squared_error_1e6: 820.7969 - psnr: 53920.6013 - val_loss: 587.3605 - val_mean_squared_error_1e6: 587.3606 - val_psnr: 52943.1706

Epoch 00115: val_psnr did not improve from 53378.00342
Epoch 116/400
249/249 [==============================] - 76s 306ms/step - loss: 858.5968 - mean_sq

249/249 [==============================] - 76s 306ms/step - loss: 740.9560 - mean_squared_error_1e6: 740.9561 - psnr: 54734.7281 - val_loss: 529.0193 - val_mean_squared_error_1e6: 529.0194 - val_psnr: 53462.0485

Epoch 00170: val_psnr did not improve from 53795.56715
Epoch 171/400
249/249 [==============================] - 76s 306ms/step - loss: 826.7653 - mean_squared_error_1e6: 826.7654 - psnr: 54450.3290 - val_loss: 524.6799 - val_mean_squared_error_1e6: 524.6800 - val_psnr: 53580.4401

Epoch 00171: val_psnr did not improve from 53795.56715
Epoch 172/400
249/249 [==============================] - 76s 305ms/step - loss: 735.5515 - mean_squared_error_1e6: 735.5516 - psnr: 54822.0201 - val_loss: 545.4094 - val_mean_squared_error_1e6: 545.4095 - val_psnr: 53385.7097

Epoch 00172: val_psnr did not improve from 53795.56715
Epoch 173/400
249/249 [==============================] - 76s 306ms/step - loss: 654.6260 - mean_squared_error_1e6: 654.6261 - psnr: 54830.3159 - val_loss: 567.5686 - va

...

249/249 [==============================] - 76s 305ms/step - loss: 701.4128 - mean_squared_error_1e6: 701.4128 - psnr: 55430.7503 - val_loss: 541.1670 - val_mean_squared_error_1e6: 541.1671 - val_psnr: 53694.1932

Epoch 00257: val_psnr did not improve from 53798.08032
Epoch 258/400
249/249 [==============================] - 76s 305ms/step - loss: 688.9652 - mean_squared_error_1e6: 688.9653 - psnr: 55361.5422 - val_loss: 552.4734 - val_mean_squared_error_1e6: 552.4735 - val_psnr: 53442.2294

Epoch 00258: val_psnr did not improve from 53798.08032
Epoch 259/400
249/249 [==============================] - 76s 305ms/step - loss: 699.7892 - mean_squared_error_1e6: 699.7892 - psnr: 55456.2307 - val_loss: 486.8658 - val_mean_squared_error_1e6: 486.8659 - val_psnr: 53836.6064

Epoch 00259: val_psnr improved from 53798.08032 to 53836.60643, saving model to .\training_models\MR-only_10\model.hdf5
Epoch 260/400
249/249 [==============================] - 76s 305ms/step - loss: 686.3515 - mean_squared


Epoch 00285: val_psnr did not improve from 53958.85876
Epoch 286/400
249/249 [==============================] - 76s 305ms/step - loss: 657.7345 - mean_squared_error_1e6: 657.7346 - psnr: 55684.8151 - val_loss: 523.2642 - val_mean_squared_error_1e6: 523.2643 - val_psnr: 53616.6402

Epoch 00286: val_psnr did not improve from 53958.85876
Epoch 287/400
249/249 [==============================] - 76s 306ms/step - loss: 658.8053 - mean_squared_error_1e6: 658.8053 - psnr: 55747.2326 - val_loss: 534.8044 - val_mean_squared_error_1e6: 534.8044 - val_psnr: 53650.1903

Epoch 00287: val_psnr did not improve from 53958.85876
Epoch 288/400
249/249 [==============================] - 76s 305ms/step - loss: 695.3103 - mean_squared_error_1e6: 695.3103 - psnr: 55762.4967 - val_loss: 515.5082 - val_mean_squared_error_1e6: 515.5083 - val_psnr: 53682.5065

Epoch 00288: val_psnr did not improve from 53958.85876
Epoch 289/400
249/249 [==============================] - 76s 305ms/step - loss: 609.3827 - mean_sq


Epoch 00314: val_psnr did not improve from 54013.24021
Epoch 315/400
249/249 [==============================] - 76s 305ms/step - loss: 530.7356 - mean_squared_error_1e6: 530.7356 - psnr: 56119.0965 - val_loss: 515.8040 - val_mean_squared_error_1e6: 515.8041 - val_psnr: 53820.9688

Epoch 00315: val_psnr did not improve from 54013.24021
Epoch 316/400
249/249 [==============================] - 76s 305ms/step - loss: 709.9228 - mean_squared_error_1e6: 709.9229 - psnr: 55701.3473 - val_loss: 522.9625 - val_mean_squared_error_1e6: 522.9626 - val_psnr: 53750.9419

Epoch 00316: val_psnr did not improve from 54013.24021
Epoch 317/400
249/249 [==============================] - 76s 305ms/step - loss: 626.5526 - mean_squared_error_1e6: 626.5526 - psnr: 55717.2098 - val_loss: 487.2130 - val_mean_squared_error_1e6: 487.2131 - val_psnr: 53883.6800

Epoch 00317: val_psnr did not improve from 54013.24021
Epoch 318/400
249/249 [==============================] - 76s 305ms/step - loss: 725.1435 - mean_sq


Epoch 00343: val_psnr did not improve from 54180.53775
Epoch 344/400
249/249 [==============================] - 76s 306ms/step - loss: 633.3496 - mean_squared_error_1e6: 633.3497 - psnr: 56046.9236 - val_loss: 511.3648 - val_mean_squared_error_1e6: 511.3649 - val_psnr: 53776.4944

Epoch 00344: val_psnr did not improve from 54180.53775
Epoch 345/400
249/249 [==============================] - 76s 305ms/step - loss: 597.9197 - mean_squared_error_1e6: 597.9197 - psnr: 56235.5357 - val_loss: 545.5341 - val_mean_squared_error_1e6: 545.5342 - val_psnr: 53536.3684

Epoch 00345: val_psnr did not improve from 54180.53775
Epoch 346/400
249/249 [==============================] - 76s 305ms/step - loss: 741.6564 - mean_squared_error_1e6: 741.6564 - psnr: 55863.5987 - val_loss: 502.4022 - val_mean_squared_error_1e6: 502.4023 - val_psnr: 53835.2444

Epoch 00346: val_psnr did not improve from 54180.53775
Epoch 347/400
249/249 [==============================] - 76s 305ms/step - loss: 698.2778 - mean_sq

249/249 [==============================] - 76s 305ms/step - loss: 660.9221 - mean_squared_error_1e6: 660.9222 - psnr: 56209.4298 - val_loss: 505.1005 - val_mean_squared_error_1e6: 505.1006 - val_psnr: 53867.9054

Epoch 00373: val_psnr did not improve from 54180.53775
Epoch 374/400
249/249 [==============================] - 76s 305ms/step - loss: 543.2690 - mean_squared_error_1e6: 543.2691 - psnr: 56572.6218 - val_loss: 510.3866 - val_mean_squared_error_1e6: 510.3867 - val_psnr: 53935.6666

Epoch 00374: val_psnr did not improve from 54180.53775
Epoch 375/400
249/249 [==============================] - 76s 306ms/step - loss: 813.9696 - mean_squared_error_1e6: 813.9697 - psnr: 55651.3193 - val_loss: 512.0594 - val_mean_squared_error_1e6: 512.0595 - val_psnr: 53908.8452

Epoch 00375: val_psnr did not improve from 54180.53775
Epoch 376/400
249/249 [==============================] - 76s 305ms/step - loss: 723.6287 - mean_squared_error_1e6: 723.6287 - psnr: 56110.2246 - val_loss: 508.2645 - va

In [9]:
depth = 4
n_fliter = 64
epochs = 400
loss = 'mse1e6'
optimizer = 'Adam'
batch_size = 5
model_type = 'PET-only' # 'PET-only', 'PET-MR', 'MR-only'
LOOCV = 17 # [0,19]

train_a_UNet(depth=depth, n_fliter=n_fliter,
             epochs=epochs, loss=loss,
             optimizer=optimizer, batch_size=batch_size,
             LOOCV=LOOCV, model_type=model_type)
print("Process is finished for "+model_type + '_' + str(LOOCV))

Epoch 1/400
249/249 [==============================] - 83s 332ms/step - loss: 13156.6028 - mean_squared_error_1e6: 13156.6030 - psnr: 44141.4702 - val_loss: 1798.7434 - val_mean_squared_error_1e6: 1798.7434 - val_psnr: 47988.0667

Epoch 00001: val_psnr improved from -inf to 47988.06667, saving model to .\training_models\PET-only_17\model.hdf5
Epoch 2/400
249/249 [==============================] - 79s 318ms/step - loss: 2441.8640 - mean_squared_error_1e6: 2441.8640 - psnr: 47310.2754 - val_loss: 1274.4145 - val_mean_squared_error_1e6: 1274.4145 - val_psnr: 49393.4991

Epoch 00002: val_psnr improved from 47988.06667 to 49393.49912, saving model to .\training_models\PET-only_17\model.hdf5
Epoch 3/400
249/249 [==============================] - 77s 309ms/step - loss: 1918.6456 - mean_squared_error_1e6: 1918.6456 - psnr: 48318.2644 - val_loss: 1007.9831 - val_mean_squared_error_1e6: 1007.9831 - val_psnr: 50551.4721

Epoch 00003: val_psnr improved from 49393.49912 to 50551.47213, saving model

249/249 [==============================] - 77s 308ms/step - loss: 725.7473 - mean_squared_error_1e6: 725.7476 - psnr: 57157.7959 - val_loss: 162.6052 - val_mean_squared_error_1e6: 162.6052 - val_psnr: 59347.8833

Epoch 00053: val_psnr did not improve from 59617.02282
Epoch 54/400
249/249 [==============================] - 77s 307ms/step - loss: 570.5729 - mean_squared_error_1e6: 570.5730 - psnr: 57577.9372 - val_loss: 157.5483 - val_mean_squared_error_1e6: 157.5483 - val_psnr: 59653.5070

Epoch 00054: val_psnr improved from 59617.02282 to 59653.50702, saving model to .\training_models\PET-only_17\model.hdf5
Epoch 55/400
249/249 [==============================] - 77s 307ms/step - loss: 605.9720 - mean_squared_error_1e6: 605.9721 - psnr: 57892.2909 - val_loss: 145.9750 - val_mean_squared_error_1e6: 145.9750 - val_psnr: 59601.4641

Epoch 00055: val_psnr did not improve from 59653.50702
Epoch 56/400
249/249 [==============================] - 77s 307ms/step - loss: 670.3468 - mean_squared_e

Epoch 81/400
249/249 [==============================] - 76s 307ms/step - loss: 551.4457 - mean_squared_error_1e6: 551.4458 - psnr: 57949.6388 - val_loss: 138.3908 - val_mean_squared_error_1e6: 138.3908 - val_psnr: 59598.0241

Epoch 00081: val_psnr did not improve from 59971.54266
Epoch 82/400
249/249 [==============================] - 76s 307ms/step - loss: 415.1275 - mean_squared_error_1e6: 415.1276 - psnr: 58671.6591 - val_loss: 142.8620 - val_mean_squared_error_1e6: 142.8620 - val_psnr: 59485.0162

Epoch 00082: val_psnr did not improve from 59971.54266
Epoch 83/400
249/249 [==============================] - 76s 307ms/step - loss: 760.3992 - mean_squared_error_1e6: 760.3993 - psnr: 57014.2403 - val_loss: 173.0530 - val_mean_squared_error_1e6: 173.0530 - val_psnr: 59303.5264

Epoch 00083: val_psnr did not improve from 59971.54266
Epoch 84/400
249/249 [==============================] - 77s 307ms/step - loss: 477.4346 - mean_squared_error_1e6: 477.4347 - psnr: 58324.2846 - val_loss: 141

Epoch 110/400
249/249 [==============================] - 76s 307ms/step - loss: 761.7520 - mean_squared_error_1e6: 761.7521 - psnr: 57531.9966 - val_loss: 156.2027 - val_mean_squared_error_1e6: 156.2027 - val_psnr: 59291.1962

Epoch 00110: val_psnr did not improve from 60023.04086
Epoch 111/400
249/249 [==============================] - 76s 307ms/step - loss: 503.7819 - mean_squared_error_1e6: 503.7820 - psnr: 58325.8565 - val_loss: 139.3832 - val_mean_squared_error_1e6: 139.3832 - val_psnr: 59921.7998

Epoch 00111: val_psnr did not improve from 60023.04086
Epoch 112/400
249/249 [==============================] - 76s 307ms/step - loss: 635.4584 - mean_squared_error_1e6: 635.4585 - psnr: 57315.2162 - val_loss: 156.0789 - val_mean_squared_error_1e6: 156.0789 - val_psnr: 59197.4117

Epoch 00112: val_psnr did not improve from 60023.04086
Epoch 113/400
249/249 [==============================] - 76s 307ms/step - loss: 665.7842 - mean_squared_error_1e6: 665.7843 - psnr: 57700.6268 - val_loss:

249/249 [==============================] - 77s 307ms/step - loss: 647.9839 - mean_squared_error_1e6: 647.9840 - psnr: 57696.5566 - val_loss: 141.8568 - val_mean_squared_error_1e6: 141.8568 - val_psnr: 59604.3639

Epoch 00168: val_psnr did not improve from 60023.04086
Epoch 169/400
249/249 [==============================] - 76s 307ms/step - loss: 505.4139 - mean_squared_error_1e6: 505.4140 - psnr: 58187.6401 - val_loss: 149.5224 - val_mean_squared_error_1e6: 149.5224 - val_psnr: 59328.3147

Epoch 00169: val_psnr did not improve from 60023.04086
Epoch 170/400
249/249 [==============================] - 76s 307ms/step - loss: 571.2637 - mean_squared_error_1e6: 571.2638 - psnr: 58406.7402 - val_loss: 168.9794 - val_mean_squared_error_1e6: 168.9794 - val_psnr: 59403.4378

Epoch 00170: val_psnr did not improve from 60023.04086
Epoch 171/400
249/249 [==============================] - 77s 307ms/step - loss: 671.5619 - mean_squared_error_1e6: 671.5619 - psnr: 57916.1633 - val_loss: 151.0178 - va

249/249 [==============================] - 76s 307ms/step - loss: 662.4759 - mean_squared_error_1e6: 662.4759 - psnr: 58153.5237 - val_loss: 164.1585 - val_mean_squared_error_1e6: 164.1585 - val_psnr: 58807.1110

Epoch 00226: val_psnr did not improve from 60023.04086
Epoch 227/400
249/249 [==============================] - 77s 307ms/step - loss: 572.0832 - mean_squared_error_1e6: 572.0833 - psnr: 58420.9771 - val_loss: 143.3727 - val_mean_squared_error_1e6: 143.3727 - val_psnr: 59412.0710

Epoch 00227: val_psnr did not improve from 60023.04086
Epoch 228/400
249/249 [==============================] - 76s 306ms/step - loss: 669.9807 - mean_squared_error_1e6: 669.9808 - psnr: 58300.8715 - val_loss: 163.2505 - val_mean_squared_error_1e6: 163.2505 - val_psnr: 58940.0809

Epoch 00228: val_psnr did not improve from 60023.04086
Epoch 229/400
249/249 [==============================] - 77s 307ms/step - loss: 642.0641 - mean_squared_error_1e6: 642.0642 - psnr: 58253.1930 - val_loss: 154.5384 - va

249/249 [==============================] - 76s 307ms/step - loss: 571.7323 - mean_squared_error_1e6: 571.7324 - psnr: 58407.8128 - val_loss: 151.0774 - val_mean_squared_error_1e6: 151.0774 - val_psnr: 59590.8501

Epoch 00284: val_psnr did not improve from 60023.04086
Epoch 285/400
249/249 [==============================] - 77s 307ms/step - loss: 639.2495 - mean_squared_error_1e6: 639.2496 - psnr: 58483.1244 - val_loss: 146.8982 - val_mean_squared_error_1e6: 146.8982 - val_psnr: 59453.2523

Epoch 00285: val_psnr did not improve from 60023.04086
Epoch 286/400
249/249 [==============================] - 76s 307ms/step - loss: 571.6797 - mean_squared_error_1e6: 571.6798 - psnr: 58297.4928 - val_loss: 156.3133 - val_mean_squared_error_1e6: 156.3133 - val_psnr: 59353.0393

Epoch 00286: val_psnr did not improve from 60023.04086
Epoch 287/400
249/249 [==============================] - 77s 307ms/step - loss: 598.9446 - mean_squared_error_1e6: 598.9447 - psnr: 58031.3126 - val_loss: 164.0172 - va

249/249 [==============================] - 76s 307ms/step - loss: 659.8306 - mean_squared_error_1e6: 659.8307 - psnr: 58458.8924 - val_loss: 150.4996 - val_mean_squared_error_1e6: 150.4996 - val_psnr: 59347.4873

Epoch 00342: val_psnr did not improve from 60023.04086
Epoch 343/400
249/249 [==============================] - 76s 307ms/step - loss: 586.7124 - mean_squared_error_1e6: 586.7125 - psnr: 58281.7725 - val_loss: 157.7270 - val_mean_squared_error_1e6: 157.7270 - val_psnr: 59179.3946

Epoch 00343: val_psnr did not improve from 60023.04086
Epoch 344/400
249/249 [==============================] - 77s 307ms/step - loss: 533.9894 - mean_squared_error_1e6: 533.9895 - psnr: 58686.2224 - val_loss: 157.7324 - val_mean_squared_error_1e6: 157.7324 - val_psnr: 59010.9748

Epoch 00344: val_psnr did not improve from 60023.04086
Epoch 345/400
249/249 [==============================] - 77s 307ms/step - loss: 511.2874 - mean_squared_error_1e6: 511.2875 - psnr: 58773.2280 - val_loss: 155.3412 - va

249/249 [==============================] - 77s 307ms/step - loss: 536.4846 - mean_squared_error_1e6: 536.4846 - psnr: 58287.8629 - val_loss: 165.6293 - val_mean_squared_error_1e6: 165.6293 - val_psnr: 59289.6788

Epoch 00400: val_psnr did not improve from 60023.04086
.\training_models\PET-only_17\model.hdf5
Prediction!
Process is finished for PET-only_17


In [9]:
depth = 4
n_fliter = 64
epochs = 400
loss = 'mse1e6'
optimizer = 'Adam'
batch_size = 5
model_type = 'PET-only' # 'PET-only', 'PET-MR', 'MR-only'
LOOCV = 16 # [0,19]

train_a_UNet(depth=depth, n_fliter=n_fliter,
             epochs=epochs, loss=loss,
             optimizer=optimizer, batch_size=batch_size,
             LOOCV=LOOCV, model_type=model_type)
print("Process is finished for "+model_type + '_' + str(LOOCV))

Epoch 1/400
249/249 [==============================] - 83s 332ms/step - loss: 13425.8201 - mean_squared_error_1e6: 13425.8202 - psnr: 43981.4716 - val_loss: 1897.2341 - val_mean_squared_error_1e6: 1897.2341 - val_psnr: 47799.4917

Epoch 00001: val_psnr improved from -inf to 47799.49166, saving model to .\training_models\PET-only_16\model.hdf5
Epoch 2/400
249/249 [==============================] - 80s 319ms/step - loss: 2456.9783 - mean_squared_error_1e6: 2456.9783 - psnr: 47239.4646 - val_loss: 1311.4930 - val_mean_squared_error_1e6: 1311.4930 - val_psnr: 49246.2482

Epoch 00002: val_psnr improved from 47799.49166 to 49246.24824, saving model to .\training_models\PET-only_16\model.hdf5
Epoch 3/400
249/249 [==============================] - 79s 316ms/step - loss: 1906.2543 - mean_squared_error_1e6: 1906.2542 - psnr: 48338.1516 - val_loss: 1063.7819 - val_mean_squared_error_1e6: 1063.7819 - val_psnr: 50265.3815

Epoch 00003: val_psnr improved from 49246.24824 to 50265.38145, saving model

249/249 [==============================] - 79s 319ms/step - loss: 687.7651 - mean_squared_error_1e6: 687.7654 - psnr: 57706.0063 - val_loss: 143.1813 - val_mean_squared_error_1e6: 143.1813 - val_psnr: 59540.5338

Epoch 00053: val_psnr did not improve from 59692.39431
Epoch 54/400
249/249 [==============================] - 79s 318ms/step - loss: 560.1124 - mean_squared_error_1e6: 560.1125 - psnr: 57817.2240 - val_loss: 152.4806 - val_mean_squared_error_1e6: 152.4806 - val_psnr: 59509.4419

Epoch 00054: val_psnr did not improve from 59692.39431
Epoch 55/400
249/249 [==============================] - 79s 318ms/step - loss: 623.2533 - mean_squared_error_1e6: 623.2534 - psnr: 57715.0305 - val_loss: 156.9874 - val_mean_squared_error_1e6: 156.9874 - val_psnr: 59468.4497

Epoch 00055: val_psnr did not improve from 59692.39431
Epoch 56/400
249/249 [==============================] - 79s 319ms/step - loss: 686.5976 - mean_squared_error_1e6: 686.5977 - psnr: 57519.7421 - val_loss: 152.3329 - val_m


Epoch 00081: val_psnr did not improve from 59906.33607
Epoch 82/400
249/249 [==============================] - 77s 309ms/step - loss: 423.3480 - mean_squared_error_1e6: 423.3481 - psnr: 58713.4394 - val_loss: 162.2275 - val_mean_squared_error_1e6: 162.2276 - val_psnr: 59129.0193

Epoch 00082: val_psnr did not improve from 59906.33607
Epoch 83/400
249/249 [==============================] - 77s 309ms/step - loss: 733.8148 - mean_squared_error_1e6: 733.8149 - psnr: 57465.1461 - val_loss: 147.4664 - val_mean_squared_error_1e6: 147.4664 - val_psnr: 59357.9753

Epoch 00083: val_psnr did not improve from 59906.33607
Epoch 84/400
249/249 [==============================] - 77s 310ms/step - loss: 480.6146 - mean_squared_error_1e6: 480.6146 - psnr: 58405.3714 - val_loss: 158.9950 - val_mean_squared_error_1e6: 158.9950 - val_psnr: 59311.7670

Epoch 00084: val_psnr did not improve from 59906.33607
Epoch 85/400
249/249 [==============================] - 77s 310ms/step - loss: 819.2714 - mean_square

249/249 [==============================] - 77s 308ms/step - loss: 637.9544 - mean_squared_error_1e6: 637.9545 - psnr: 58262.3496 - val_loss: 157.3803 - val_mean_squared_error_1e6: 157.3803 - val_psnr: 59667.9871

Epoch 00139: val_psnr did not improve from 59924.71151
Epoch 140/400
249/249 [==============================] - 77s 308ms/step - loss: 516.4068 - mean_squared_error_1e6: 516.4069 - psnr: 58697.4688 - val_loss: 143.1070 - val_mean_squared_error_1e6: 143.1070 - val_psnr: 59777.5406

Epoch 00140: val_psnr did not improve from 59924.71151
Epoch 141/400
249/249 [==============================] - 77s 308ms/step - loss: 606.6180 - mean_squared_error_1e6: 606.6182 - psnr: 57992.8086 - val_loss: 140.0992 - val_mean_squared_error_1e6: 140.0992 - val_psnr: 59496.6458

Epoch 00141: val_psnr did not improve from 59924.71151
Epoch 142/400
249/249 [==============================] - 77s 308ms/step - loss: 613.4465 - mean_squared_error_1e6: 613.4466 - psnr: 58120.4805 - val_loss: 149.3864 - va

249/249 [==============================] - 77s 311ms/step - loss: 661.8628 - mean_squared_error_1e6: 661.8629 - psnr: 58060.9119 - val_loss: 156.6794 - val_mean_squared_error_1e6: 156.6794 - val_psnr: 59349.1261

Epoch 00197: val_psnr did not improve from 59955.88101
Epoch 198/400
249/249 [==============================] - 77s 311ms/step - loss: 596.9172 - mean_squared_error_1e6: 596.9173 - psnr: 58500.7640 - val_loss: 159.4049 - val_mean_squared_error_1e6: 159.4049 - val_psnr: 59807.7835

Epoch 00198: val_psnr did not improve from 59955.88101
Epoch 199/400
249/249 [==============================] - 77s 311ms/step - loss: 548.5740 - mean_squared_error_1e6: 548.5741 - psnr: 58359.4242 - val_loss: 173.2324 - val_mean_squared_error_1e6: 173.2324 - val_psnr: 59297.6092

Epoch 00199: val_psnr did not improve from 59955.88101
Epoch 200/400
249/249 [==============================] - 78s 312ms/step - loss: 743.1172 - mean_squared_error_1e6: 743.1174 - psnr: 58189.8203 - val_loss: 154.1109 - va

249/249 [==============================] - 77s 308ms/step - loss: 680.0561 - mean_squared_error_1e6: 680.0562 - psnr: 58098.6912 - val_loss: 168.3773 - val_mean_squared_error_1e6: 168.3773 - val_psnr: 59066.4055

Epoch 00226: val_psnr did not improve from 60007.71186
Epoch 227/400
249/249 [==============================] - 77s 309ms/step - loss: 580.9365 - mean_squared_error_1e6: 580.9366 - psnr: 58471.2753 - val_loss: 160.8586 - val_mean_squared_error_1e6: 160.8586 - val_psnr: 59401.3683

Epoch 00227: val_psnr did not improve from 60007.71186
Epoch 228/400
249/249 [==============================] - 77s 309ms/step - loss: 673.3583 - mean_squared_error_1e6: 673.3584 - psnr: 58389.9626 - val_loss: 166.6540 - val_mean_squared_error_1e6: 166.6540 - val_psnr: 59203.3444

Epoch 00228: val_psnr did not improve from 60007.71186
Epoch 229/400
249/249 [==============================] - 77s 309ms/step - loss: 638.2343 - mean_squared_error_1e6: 638.2344 - psnr: 58420.8576 - val_loss: 148.4567 - va

249/249 [==============================] - 77s 308ms/step - loss: 567.6610 - mean_squared_error_1e6: 567.6611 - psnr: 58607.2698 - val_loss: 159.2308 - val_mean_squared_error_1e6: 159.2308 - val_psnr: 59433.2111

Epoch 00284: val_psnr did not improve from 60007.71186
Epoch 285/400
249/249 [==============================] - 77s 308ms/step - loss: 635.9521 - mean_squared_error_1e6: 635.9522 - psnr: 58665.0815 - val_loss: 140.1482 - val_mean_squared_error_1e6: 140.1482 - val_psnr: 59701.4540

Epoch 00285: val_psnr did not improve from 60007.71186
Epoch 286/400
249/249 [==============================] - 77s 308ms/step - loss: 556.4069 - mean_squared_error_1e6: 556.4069 - psnr: 58548.2069 - val_loss: 177.1317 - val_mean_squared_error_1e6: 177.1317 - val_psnr: 59283.6189

Epoch 00286: val_psnr did not improve from 60007.71186
Epoch 287/400
249/249 [==============================] - 77s 308ms/step - loss: 574.3067 - mean_squared_error_1e6: 574.3068 - psnr: 58526.1747 - val_loss: 150.7149 - va

249/249 [==============================] - 77s 308ms/step - loss: 585.6174 - mean_squared_error_1e6: 585.6174 - psnr: 58644.3639 - val_loss: 204.6174 - val_mean_squared_error_1e6: 204.6174 - val_psnr: 59628.7810

Epoch 00313: val_psnr did not improve from 60151.66929
Epoch 314/400
249/249 [==============================] - 77s 308ms/step - loss: 533.5149 - mean_squared_error_1e6: 533.5150 - psnr: 58686.2858 - val_loss: 142.8863 - val_mean_squared_error_1e6: 142.8863 - val_psnr: 59709.2512

Epoch 00314: val_psnr did not improve from 60151.66929
Epoch 315/400
249/249 [==============================] - 77s 308ms/step - loss: 433.4259 - mean_squared_error_1e6: 433.4259 - psnr: 58993.7148 - val_loss: 156.1458 - val_mean_squared_error_1e6: 156.1458 - val_psnr: 59417.2822

Epoch 00315: val_psnr did not improve from 60151.66929
Epoch 316/400
249/249 [==============================] - 77s 308ms/step - loss: 651.7957 - mean_squared_error_1e6: 651.7957 - psnr: 57990.7513 - val_loss: 162.1114 - va

249/249 [==============================] - 77s 308ms/step - loss: 650.0738 - mean_squared_error_1e6: 650.0738 - psnr: 58849.6418 - val_loss: 147.3245 - val_mean_squared_error_1e6: 147.3245 - val_psnr: 59479.2306

Epoch 00371: val_psnr did not improve from 60151.66929
Epoch 372/400
249/249 [==============================] - 77s 308ms/step - loss: 548.2795 - mean_squared_error_1e6: 548.2795 - psnr: 59077.3336 - val_loss: 138.9493 - val_mean_squared_error_1e6: 138.9493 - val_psnr: 59816.6396

Epoch 00372: val_psnr did not improve from 60151.66929
Epoch 373/400
249/249 [==============================] - 77s 308ms/step - loss: 585.2968 - mean_squared_error_1e6: 585.2969 - psnr: 58692.8857 - val_loss: 148.2503 - val_mean_squared_error_1e6: 148.2503 - val_psnr: 59378.9897

Epoch 00373: val_psnr did not improve from 60151.66929
Epoch 374/400
249/249 [==============================] - 77s 308ms/step - loss: 456.5176 - mean_squared_error_1e6: 456.5177 - psnr: 59192.7277 - val_loss: 155.2173 - va

In [9]:
depth = 4
n_fliter = 64
epochs = 400
loss = 'mse1e6'
optimizer = 'Adam'
batch_size = 5
model_type = 'PET-only' # 'PET-only', 'PET-MR', 'MR-only'
LOOCV = 15 # [0,19]

train_a_UNet(depth=depth, n_fliter=n_fliter,
             epochs=epochs, loss=loss,
             optimizer=optimizer, batch_size=batch_size,
             LOOCV=LOOCV, model_type=model_type)
print("Process is finished for "+model_type + '_' + str(LOOCV))

Epoch 1/400
249/249 [==============================] - 88s 353ms/step - loss: 13467.7755 - mean_squared_error_1e6: 13467.7754 - psnr: 43988.1530 - val_loss: 1869.4243 - val_mean_squared_error_1e6: 1869.4244 - val_psnr: 47864.4376

Epoch 00001: val_psnr improved from -inf to 47864.43763, saving model to .\training_models\PET-only_15\model.hdf5
Epoch 2/400
249/249 [==============================] - 80s 319ms/step - loss: 2467.1638 - mean_squared_error_1e6: 2467.1638 - psnr: 47227.1176 - val_loss: 1314.0235 - val_mean_squared_error_1e6: 1314.0235 - val_psnr: 49240.2461

Epoch 00002: val_psnr improved from 47864.43763 to 49240.24614, saving model to .\training_models\PET-only_15\model.hdf5
Epoch 3/400
249/249 [==============================] - 79s 316ms/step - loss: 1920.5942 - mean_squared_error_1e6: 1920.5942 - psnr: 48317.5112 - val_loss: 1014.4265 - val_mean_squared_error_1e6: 1014.4265 - val_psnr: 50500.6616

Epoch 00003: val_psnr improved from 49240.24614 to 50500.66156, saving model


Epoch 00052: val_psnr improved from 59606.11609 to 59651.70488, saving model to .\training_models\PET-only_15\model.hdf5
Epoch 53/400
249/249 [==============================] - 78s 313ms/step - loss: 697.1273 - mean_squared_error_1e6: 697.1275 - psnr: 57517.6646 - val_loss: 143.0358 - val_mean_squared_error_1e6: 143.0358 - val_psnr: 59515.1615

Epoch 00053: val_psnr did not improve from 59651.70488
Epoch 54/400
249/249 [==============================] - 78s 313ms/step - loss: 570.0394 - mean_squared_error_1e6: 570.0395 - psnr: 57623.4221 - val_loss: 150.8798 - val_mean_squared_error_1e6: 150.8798 - val_psnr: 59545.0600

Epoch 00054: val_psnr did not improve from 59651.70488
Epoch 55/400
249/249 [==============================] - 78s 313ms/step - loss: 615.0919 - mean_squared_error_1e6: 615.0919 - psnr: 57750.6192 - val_loss: 155.7402 - val_mean_squared_error_1e6: 155.7402 - val_psnr: 59277.0882

Epoch 00055: val_psnr did not improve from 59651.70488
Epoch 56/400
249/249 [=============


Epoch 00081: val_psnr did not improve from 59961.33287
Epoch 82/400
249/249 [==============================] - 77s 308ms/step - loss: 440.7064 - mean_squared_error_1e6: 440.7065 - psnr: 58408.4226 - val_loss: 166.6640 - val_mean_squared_error_1e6: 166.6640 - val_psnr: 59285.6113

Epoch 00082: val_psnr did not improve from 59961.33287
Epoch 83/400
249/249 [==============================] - 77s 308ms/step - loss: 760.9878 - mean_squared_error_1e6: 760.9879 - psnr: 57037.7006 - val_loss: 175.8501 - val_mean_squared_error_1e6: 175.8501 - val_psnr: 58926.3470

Epoch 00083: val_psnr did not improve from 59961.33287
Epoch 84/400
249/249 [==============================] - 77s 308ms/step - loss: 482.5566 - mean_squared_error_1e6: 482.5567 - psnr: 58299.0285 - val_loss: 135.8881 - val_mean_squared_error_1e6: 135.8881 - val_psnr: 59714.8308

Epoch 00084: val_psnr did not improve from 59961.33287
Epoch 85/400
249/249 [==============================] - 77s 308ms/step - loss: 716.4193 - mean_square

249/249 [==============================] - 77s 308ms/step - loss: 508.0211 - mean_squared_error_1e6: 508.0212 - psnr: 58311.1753 - val_loss: 140.9722 - val_mean_squared_error_1e6: 140.9722 - val_psnr: 59813.3701

Epoch 00111: val_psnr did not improve from 59961.33287
Epoch 112/400
249/249 [==============================] - 77s 308ms/step - loss: 638.9560 - mean_squared_error_1e6: 638.9561 - psnr: 57340.1147 - val_loss: 149.4685 - val_mean_squared_error_1e6: 149.4685 - val_psnr: 59302.3470

Epoch 00112: val_psnr did not improve from 59961.33287
Epoch 113/400
249/249 [==============================] - 77s 308ms/step - loss: 629.4524 - mean_squared_error_1e6: 629.4525 - psnr: 57939.9957 - val_loss: 151.9023 - val_mean_squared_error_1e6: 151.9023 - val_psnr: 59404.2888

Epoch 00113: val_psnr did not improve from 59961.33287
Epoch 114/400
249/249 [==============================] - 77s 308ms/step - loss: 525.3938 - mean_squared_error_1e6: 525.3938 - psnr: 58066.1259 - val_loss: 153.7161 - va

249/249 [==============================] - 77s 308ms/step - loss: 534.6872 - mean_squared_error_1e6: 534.6872 - psnr: 57898.2917 - val_loss: 197.9411 - val_mean_squared_error_1e6: 197.9411 - val_psnr: 59290.6356

Epoch 00169: val_psnr did not improve from 60052.54213
Epoch 170/400
249/249 [==============================] - 77s 308ms/step - loss: 574.4276 - mean_squared_error_1e6: 574.4277 - psnr: 58450.8386 - val_loss: 145.8469 - val_mean_squared_error_1e6: 145.8469 - val_psnr: 59555.3405

Epoch 00170: val_psnr did not improve from 60052.54213
Epoch 171/400
249/249 [==============================] - 77s 308ms/step - loss: 673.8450 - mean_squared_error_1e6: 673.8451 - psnr: 57912.1655 - val_loss: 144.2009 - val_mean_squared_error_1e6: 144.2009 - val_psnr: 59508.2999

Epoch 00171: val_psnr did not improve from 60052.54213
Epoch 172/400
249/249 [==============================] - 77s 308ms/step - loss: 580.7845 - mean_squared_error_1e6: 580.7846 - psnr: 58460.2442 - val_loss: 145.6697 - va

249/249 [==============================] - 77s 308ms/step - loss: 576.3402 - mean_squared_error_1e6: 576.3402 - psnr: 58386.6534 - val_loss: 143.0107 - val_mean_squared_error_1e6: 143.0107 - val_psnr: 59327.4105

Epoch 00227: val_psnr did not improve from 60052.54213
Epoch 228/400
249/249 [==============================] - 77s 308ms/step - loss: 683.7762 - mean_squared_error_1e6: 683.7763 - psnr: 58109.3050 - val_loss: 181.0246 - val_mean_squared_error_1e6: 181.0246 - val_psnr: 58851.5026

Epoch 00228: val_psnr did not improve from 60052.54213
Epoch 229/400
249/249 [==============================] - 77s 308ms/step - loss: 644.4225 - mean_squared_error_1e6: 644.4225 - psnr: 58206.9167 - val_loss: 153.2308 - val_mean_squared_error_1e6: 153.2308 - val_psnr: 59400.5895

Epoch 00229: val_psnr did not improve from 60052.54213
Epoch 230/400
249/249 [==============================] - 77s 308ms/step - loss: 658.1377 - mean_squared_error_1e6: 658.1378 - psnr: 58063.7112 - val_loss: 181.4165 - va

249/249 [==============================] - 77s 308ms/step - loss: 644.2536 - mean_squared_error_1e6: 644.2537 - psnr: 58430.3878 - val_loss: 146.2366 - val_mean_squared_error_1e6: 146.2366 - val_psnr: 59438.0891

Epoch 00285: val_psnr did not improve from 60052.54213
Epoch 286/400
249/249 [==============================] - 77s 309ms/step - loss: 555.0163 - mean_squared_error_1e6: 555.0163 - psnr: 58451.9928 - val_loss: 149.9540 - val_mean_squared_error_1e6: 149.9540 - val_psnr: 59167.4616

Epoch 00286: val_psnr did not improve from 60052.54213
Epoch 287/400
249/249 [==============================] - 77s 308ms/step - loss: 581.6560 - mean_squared_error_1e6: 581.6561 - psnr: 58298.1417 - val_loss: 160.6130 - val_mean_squared_error_1e6: 160.6130 - val_psnr: 58883.5338

Epoch 00287: val_psnr did not improve from 60052.54213
Epoch 288/400
249/249 [==============================] - 77s 308ms/step - loss: 618.7693 - mean_squared_error_1e6: 618.7694 - psnr: 58186.0300 - val_loss: 153.3415 - va

249/249 [==============================] - 77s 308ms/step - loss: 574.4439 - mean_squared_error_1e6: 574.4439 - psnr: 58440.2955 - val_loss: 160.4154 - val_mean_squared_error_1e6: 160.4154 - val_psnr: 59240.3462

Epoch 00343: val_psnr did not improve from 60052.54213
Epoch 344/400
249/249 [==============================] - 77s 308ms/step - loss: 537.8595 - mean_squared_error_1e6: 537.8595 - psnr: 58657.4735 - val_loss: 150.5939 - val_mean_squared_error_1e6: 150.5939 - val_psnr: 59221.7762

Epoch 00344: val_psnr did not improve from 60052.54213
Epoch 345/400
249/249 [==============================] - 77s 308ms/step - loss: 511.3205 - mean_squared_error_1e6: 511.3206 - psnr: 58774.9040 - val_loss: 147.2444 - val_mean_squared_error_1e6: 147.2444 - val_psnr: 59521.0046

Epoch 00345: val_psnr did not improve from 60052.54213
Epoch 346/400
249/249 [==============================] - 77s 308ms/step - loss: 651.8289 - mean_squared_error_1e6: 651.8290 - psnr: 58290.7983 - val_loss: 156.0518 - va

Prediction!
Process is finished for PET-only_15


In [10]:
depth = 4
n_fliter = 64
epochs = 400
loss = 'mse1e6'
optimizer = 'Adam'
batch_size = 5
model_type = 'PET-only' # 'PET-only', 'PET-MR', 'MR-only'
LOOCV = 14 # [0,19]

train_a_UNet(depth=depth, n_fliter=n_fliter,
             epochs=epochs, loss=loss,
             optimizer=optimizer, batch_size=batch_size,
             LOOCV=LOOCV, model_type=model_type)
print("Process is finished for "+model_type + '_' + str(LOOCV))

Epoch 1/400
249/249 [==============================] - 83s 333ms/step - loss: 13386.4904 - mean_squared_error_1e6: 13386.4901 - psnr: 43949.2551 - val_loss: 1895.7059 - val_mean_squared_error_1e6: 1895.7059 - val_psnr: 47801.3485

Epoch 00001: val_psnr improved from -inf to 47801.34853, saving model to .\training_models\PET-only_14\model.hdf5
Epoch 2/400
249/249 [==============================] - 79s 319ms/step - loss: 2462.9963 - mean_squared_error_1e6: 2462.9963 - psnr: 47178.5261 - val_loss: 1301.8737 - val_mean_squared_error_1e6: 1301.8737 - val_psnr: 49267.7389

Epoch 00002: val_psnr improved from 47801.34853 to 49267.73885, saving model to .\training_models\PET-only_14\model.hdf5
Epoch 3/400
249/249 [==============================] - 78s 315ms/step - loss: 1918.9726 - mean_squared_error_1e6: 1918.9726 - psnr: 48283.2952 - val_loss: 1014.5774 - val_mean_squared_error_1e6: 1014.5774 - val_psnr: 50499.0775

Epoch 00003: val_psnr improved from 49267.73885 to 50499.07747, saving model

249/249 [==============================] - 79s 315ms/step - loss: 692.8602 - mean_squared_error_1e6: 692.8604 - psnr: 57570.3112 - val_loss: 145.1238 - val_mean_squared_error_1e6: 145.1238 - val_psnr: 59443.2571

Epoch 00053: val_psnr did not improve from 59735.08265
Epoch 54/400
249/249 [==============================] - 79s 316ms/step - loss: 553.8404 - mean_squared_error_1e6: 553.8405 - psnr: 57843.9177 - val_loss: 139.2027 - val_mean_squared_error_1e6: 139.2027 - val_psnr: 59794.2812

Epoch 00054: val_psnr improved from 59735.08265 to 59794.28121, saving model to .\training_models\PET-only_14\model.hdf5
Epoch 55/400
249/249 [==============================] - 78s 314ms/step - loss: 613.9953 - mean_squared_error_1e6: 613.9955 - psnr: 57730.5793 - val_loss: 148.7810 - val_mean_squared_error_1e6: 148.7810 - val_psnr: 59563.4009

Epoch 00055: val_psnr did not improve from 59794.28121
Epoch 56/400
249/249 [==============================] - 77s 308ms/step - loss: 670.9021 - mean_squared_e


Epoch 00081: val_psnr did not improve from 59874.51374
Epoch 82/400
249/249 [==============================] - 77s 308ms/step - loss: 423.8320 - mean_squared_error_1e6: 423.8322 - psnr: 58568.7175 - val_loss: 140.7710 - val_mean_squared_error_1e6: 140.7710 - val_psnr: 59567.5194

Epoch 00082: val_psnr did not improve from 59874.51374
Epoch 83/400
249/249 [==============================] - 77s 308ms/step - loss: 749.7160 - mean_squared_error_1e6: 749.7162 - psnr: 57223.4284 - val_loss: 162.8882 - val_mean_squared_error_1e6: 162.8882 - val_psnr: 59222.7233

Epoch 00083: val_psnr did not improve from 59874.51374
Epoch 84/400
249/249 [==============================] - 77s 308ms/step - loss: 480.2193 - mean_squared_error_1e6: 480.2194 - psnr: 58328.2365 - val_loss: 135.3048 - val_mean_squared_error_1e6: 135.3048 - val_psnr: 59750.0880

Epoch 00084: val_psnr did not improve from 59874.51374
Epoch 85/400
249/249 [==============================] - 77s 308ms/step - loss: 713.8463 - mean_square


Epoch 00110: val_psnr did not improve from 59915.11008
Epoch 111/400
249/249 [==============================] - 77s 308ms/step - loss: 526.0216 - mean_squared_error_1e6: 526.0217 - psnr: 57961.3727 - val_loss: 159.4536 - val_mean_squared_error_1e6: 159.4536 - val_psnr: 59629.2245

Epoch 00111: val_psnr did not improve from 59915.11008
Epoch 112/400
249/249 [==============================] - 77s 308ms/step - loss: 624.7105 - mean_squared_error_1e6: 624.7106 - psnr: 57493.2293 - val_loss: 144.1186 - val_mean_squared_error_1e6: 144.1186 - val_psnr: 59393.5467

Epoch 00112: val_psnr did not improve from 59915.11008
Epoch 113/400
249/249 [==============================] - 77s 308ms/step - loss: 626.6446 - mean_squared_error_1e6: 626.6447 - psnr: 57925.9106 - val_loss: 155.2460 - val_mean_squared_error_1e6: 155.2460 - val_psnr: 59265.0017

Epoch 00113: val_psnr did not improve from 59915.11008
Epoch 114/400
249/249 [==============================] - 77s 308ms/step - loss: 523.4113 - mean_sq

Epoch 140/400
249/249 [==============================] - 77s 308ms/step - loss: 531.3072 - mean_squared_error_1e6: 531.3072 - psnr: 58477.4777 - val_loss: 207.0165 - val_mean_squared_error_1e6: 207.0165 - val_psnr: 59349.8605

Epoch 00140: val_psnr did not improve from 59915.11008
Epoch 141/400
249/249 [==============================] - 77s 308ms/step - loss: 617.6236 - mean_squared_error_1e6: 617.6237 - psnr: 57721.2461 - val_loss: 145.4536 - val_mean_squared_error_1e6: 145.4536 - val_psnr: 59244.9088

Epoch 00141: val_psnr did not improve from 59915.11008
Epoch 142/400
249/249 [==============================] - 77s 307ms/step - loss: 620.5136 - mean_squared_error_1e6: 620.5138 - psnr: 57905.8078 - val_loss: 154.9894 - val_mean_squared_error_1e6: 154.9894 - val_psnr: 59129.8809

Epoch 00142: val_psnr did not improve from 59915.11008
Epoch 143/400
249/249 [==============================] - 77s 307ms/step - loss: 533.2636 - mean_squared_error_1e6: 533.2637 - psnr: 58264.8368 - val_loss:

249/249 [==============================] - 77s 307ms/step - loss: 602.0484 - mean_squared_error_1e6: 602.0485 - psnr: 58283.3101 - val_loss: 172.8282 - val_mean_squared_error_1e6: 172.8282 - val_psnr: 59610.0790

Epoch 00198: val_psnr did not improve from 59915.11008
Epoch 199/400
249/249 [==============================] - 77s 308ms/step - loss: 563.7851 - mean_squared_error_1e6: 563.7852 - psnr: 58003.9777 - val_loss: 180.5081 - val_mean_squared_error_1e6: 180.5081 - val_psnr: 59068.3818

Epoch 00199: val_psnr did not improve from 59915.11008
Epoch 200/400
249/249 [==============================] - 77s 308ms/step - loss: 757.0603 - mean_squared_error_1e6: 757.0604 - psnr: 57869.3971 - val_loss: 169.0176 - val_mean_squared_error_1e6: 169.0177 - val_psnr: 58958.4353

Epoch 00200: val_psnr did not improve from 59915.11008
Epoch 201/400
249/249 [==============================] - 77s 308ms/step - loss: 507.9130 - mean_squared_error_1e6: 507.9130 - psnr: 58382.6289 - val_loss: 150.9378 - va

249/249 [==============================] - 77s 308ms/step - loss: 624.2936 - mean_squared_error_1e6: 624.2937 - psnr: 58198.2962 - val_loss: 169.8775 - val_mean_squared_error_1e6: 169.8775 - val_psnr: 59207.7726

Epoch 00256: val_psnr did not improve from 59915.11008
Epoch 257/400
249/249 [==============================] - 77s 308ms/step - loss: 582.6050 - mean_squared_error_1e6: 582.6051 - psnr: 58485.1520 - val_loss: 157.0159 - val_mean_squared_error_1e6: 157.0159 - val_psnr: 59222.6001

Epoch 00257: val_psnr did not improve from 59915.11008
Epoch 258/400
249/249 [==============================] - 77s 308ms/step - loss: 615.4810 - mean_squared_error_1e6: 615.4811 - psnr: 58228.2210 - val_loss: 158.4302 - val_mean_squared_error_1e6: 158.4302 - val_psnr: 59151.7842

Epoch 00258: val_psnr did not improve from 59915.11008
Epoch 259/400
249/249 [==============================] - 77s 308ms/step - loss: 601.0709 - mean_squared_error_1e6: 601.0710 - psnr: 58281.2924 - val_loss: 173.9163 - va

249/249 [==============================] - 77s 307ms/step - loss: 551.9149 - mean_squared_error_1e6: 551.9150 - psnr: 58310.5383 - val_loss: 193.8806 - val_mean_squared_error_1e6: 193.8806 - val_psnr: 59427.6102

Epoch 00314: val_psnr did not improve from 59915.11008
Epoch 315/400
249/249 [==============================] - 77s 307ms/step - loss: 438.7426 - mean_squared_error_1e6: 438.7426 - psnr: 58823.4995 - val_loss: 152.5320 - val_mean_squared_error_1e6: 152.5320 - val_psnr: 59424.8596

Epoch 00315: val_psnr did not improve from 59915.11008
Epoch 316/400
249/249 [==============================] - 77s 308ms/step - loss: 614.7439 - mean_squared_error_1e6: 614.7440 - psnr: 58279.8637 - val_loss: 155.6509 - val_mean_squared_error_1e6: 155.6509 - val_psnr: 59141.7116

Epoch 00316: val_psnr did not improve from 59915.11008
Epoch 317/400
249/249 [==============================] - 77s 308ms/step - loss: 533.2119 - mean_squared_error_1e6: 533.2120 - psnr: 58287.2594 - val_loss: 154.3662 - va

249/249 [==============================] - 77s 308ms/step - loss: 553.8862 - mean_squared_error_1e6: 553.8862 - psnr: 58879.4758 - val_loss: 138.3713 - val_mean_squared_error_1e6: 138.3713 - val_psnr: 59842.4661

Epoch 00372: val_psnr did not improve from 59928.26905
Epoch 373/400
249/249 [==============================] - 77s 308ms/step - loss: 596.4887 - mean_squared_error_1e6: 596.4887 - psnr: 58513.5329 - val_loss: 162.8572 - val_mean_squared_error_1e6: 162.8572 - val_psnr: 59278.4023

Epoch 00373: val_psnr did not improve from 59928.26905
Epoch 374/400
249/249 [==============================] - 77s 307ms/step - loss: 557.8338 - mean_squared_error_1e6: 557.8339 - psnr: 58718.1566 - val_loss: 171.3371 - val_mean_squared_error_1e6: 171.3371 - val_psnr: 59558.6316

Epoch 00374: val_psnr did not improve from 59928.26905
Epoch 375/400
249/249 [==============================] - 77s 308ms/step - loss: 708.5472 - mean_squared_error_1e6: 708.5472 - psnr: 58289.6006 - val_loss: 146.9286 - va

Prediction!
Process is finished for PET-only_14


In [9]:
depth = 4
n_fliter = 64
epochs = 400
loss = 'mse1e6'
optimizer = 'Adam'
batch_size = 5
model_type = 'PET-only' # 'PET-only', 'PET-MR', 'MR-only'
LOOCV = 13 # [0,19]

train_a_UNet(depth=depth, n_fliter=n_fliter,
             epochs=epochs, loss=loss,
             optimizer=optimizer, batch_size=batch_size,
             LOOCV=LOOCV, model_type=model_type)
print("Process is finished for "+model_type + '_' + str(LOOCV))

Epoch 1/400
249/249 [==============================] - 84s 337ms/step - loss: 13307.3879 - mean_squared_error_1e6: 13307.3878 - psnr: 44023.7253 - val_loss: 1888.0304 - val_mean_squared_error_1e6: 1888.0304 - val_psnr: 47855.4731

Epoch 00001: val_psnr improved from -inf to 47855.47314, saving model to .\training_models\PET-only_13\model.hdf5
Epoch 2/400
249/249 [==============================] - 80s 321ms/step - loss: 2448.3916 - mean_squared_error_1e6: 2448.3916 - psnr: 47255.6464 - val_loss: 1305.1773 - val_mean_squared_error_1e6: 1305.1773 - val_psnr: 49255.5379

Epoch 00002: val_psnr improved from 47855.47314 to 49255.53788, saving model to .\training_models\PET-only_13\model.hdf5
Epoch 3/400
249/249 [==============================] - 79s 315ms/step - loss: 1904.3647 - mean_squared_error_1e6: 1904.3647 - psnr: 48358.9200 - val_loss: 1017.4441 - val_mean_squared_error_1e6: 1017.4441 - val_psnr: 50492.5156

Epoch 00003: val_psnr improved from 49255.53788 to 50492.51562, saving model

249/249 [==============================] - 77s 309ms/step - loss: 691.9480 - mean_squared_error_1e6: 691.9481 - psnr: 57597.5724 - val_loss: 143.6046 - val_mean_squared_error_1e6: 143.6047 - val_psnr: 59512.1642

Epoch 00053: val_psnr did not improve from 59663.22994
Epoch 54/400
249/249 [==============================] - 77s 308ms/step - loss: 561.4046 - mean_squared_error_1e6: 561.4048 - psnr: 57745.3235 - val_loss: 151.7147 - val_mean_squared_error_1e6: 151.7147 - val_psnr: 59697.9630

Epoch 00054: val_psnr improved from 59663.22994 to 59697.96296, saving model to .\training_models\PET-only_13\model.hdf5
Epoch 55/400
249/249 [==============================] - 77s 308ms/step - loss: 615.4263 - mean_squared_error_1e6: 615.4264 - psnr: 57724.2382 - val_loss: 145.1409 - val_mean_squared_error_1e6: 145.1409 - val_psnr: 59702.2598

Epoch 00055: val_psnr improved from 59697.96296 to 59702.25979, saving model to .\training_models\PET-only_13\model.hdf5
Epoch 56/400
249/249 [================


Epoch 00109: val_psnr did not improve from 60014.82957
Epoch 110/400
249/249 [==============================] - 77s 309ms/step - loss: 767.6632 - mean_squared_error_1e6: 767.6633 - psnr: 57448.1090 - val_loss: 154.1005 - val_mean_squared_error_1e6: 154.1005 - val_psnr: 59342.6968

Epoch 00110: val_psnr did not improve from 60014.82957
Epoch 111/400
249/249 [==============================] - 77s 308ms/step - loss: 528.7663 - mean_squared_error_1e6: 528.7664 - psnr: 58023.0973 - val_loss: 150.4155 - val_mean_squared_error_1e6: 150.4155 - val_psnr: 59671.0817

Epoch 00111: val_psnr did not improve from 60014.82957
Epoch 112/400
249/249 [==============================] - 77s 308ms/step - loss: 640.9599 - mean_squared_error_1e6: 640.9600 - psnr: 57257.9670 - val_loss: 144.2213 - val_mean_squared_error_1e6: 144.2213 - val_psnr: 59557.7667

Epoch 00112: val_psnr did not improve from 60014.82957
Epoch 113/400
249/249 [==============================] - 77s 308ms/step - loss: 663.5571 - mean_sq

Epoch 139/400
249/249 [==============================] - 77s 308ms/step - loss: 634.8438 - mean_squared_error_1e6: 634.8439 - psnr: 58176.0613 - val_loss: 143.4368 - val_mean_squared_error_1e6: 143.4368 - val_psnr: 59627.6385

Epoch 00139: val_psnr did not improve from 60014.82957
Epoch 140/400
249/249 [==============================] - 77s 308ms/step - loss: 525.0859 - mean_squared_error_1e6: 525.0859 - psnr: 58499.3062 - val_loss: 142.1365 - val_mean_squared_error_1e6: 142.1365 - val_psnr: 59788.0672

Epoch 00140: val_psnr did not improve from 60014.82957
Epoch 141/400
249/249 [==============================] - 77s 308ms/step - loss: 631.5394 - mean_squared_error_1e6: 631.5395 - psnr: 57542.8705 - val_loss: 163.6021 - val_mean_squared_error_1e6: 163.6021 - val_psnr: 59400.9322

Epoch 00141: val_psnr did not improve from 60014.82957
Epoch 142/400
249/249 [==============================] - 77s 308ms/step - loss: 624.0827 - mean_squared_error_1e6: 624.0828 - psnr: 57853.9302 - val_loss:

249/249 [==============================] - 77s 308ms/step - loss: 628.8757 - mean_squared_error_1e6: 628.8758 - psnr: 58163.6446 - val_loss: 149.2635 - val_mean_squared_error_1e6: 149.2635 - val_psnr: 59412.4532

Epoch 00197: val_psnr did not improve from 60014.82957
Epoch 198/400
249/249 [==============================] - 77s 308ms/step - loss: 604.9281 - mean_squared_error_1e6: 604.9281 - psnr: 58229.3691 - val_loss: 165.1828 - val_mean_squared_error_1e6: 165.1828 - val_psnr: 59956.6305

Epoch 00198: val_psnr did not improve from 60014.82957
Epoch 199/400
249/249 [==============================] - 77s 308ms/step - loss: 542.7544 - mean_squared_error_1e6: 542.7545 - psnr: 58201.8800 - val_loss: 169.8547 - val_mean_squared_error_1e6: 169.8547 - val_psnr: 59461.1189

Epoch 00199: val_psnr did not improve from 60014.82957
Epoch 200/400
249/249 [==============================] - 77s 308ms/step - loss: 759.7864 - mean_squared_error_1e6: 759.7865 - psnr: 57813.3695 - val_loss: 166.3656 - va

249/249 [==============================] - 77s 308ms/step - loss: 543.8974 - mean_squared_error_1e6: 543.8974 - psnr: 58412.4989 - val_loss: 161.0860 - val_mean_squared_error_1e6: 161.0860 - val_psnr: 59002.6215

Epoch 00255: val_psnr did not improve from 60014.82957
Epoch 256/400
249/249 [==============================] - 77s 308ms/step - loss: 610.6233 - mean_squared_error_1e6: 610.6233 - psnr: 58309.0588 - val_loss: 152.3150 - val_mean_squared_error_1e6: 152.3150 - val_psnr: 59326.4712

Epoch 00256: val_psnr did not improve from 60014.82957
Epoch 257/400
249/249 [==============================] - 77s 308ms/step - loss: 587.3383 - mean_squared_error_1e6: 587.3383 - psnr: 58372.6547 - val_loss: 155.2282 - val_mean_squared_error_1e6: 155.2282 - val_psnr: 59218.6561

Epoch 00257: val_psnr did not improve from 60014.82957
Epoch 258/400
249/249 [==============================] - 77s 308ms/step - loss: 613.3379 - mean_squared_error_1e6: 613.3380 - psnr: 58186.6707 - val_loss: 162.7785 - va

249/249 [==============================] - 77s 308ms/step - loss: 583.2847 - mean_squared_error_1e6: 583.2848 - psnr: 58530.6113 - val_loss: 146.3146 - val_mean_squared_error_1e6: 146.3146 - val_psnr: 59492.7067

Epoch 00313: val_psnr did not improve from 60014.82957
Epoch 314/400
249/249 [==============================] - 77s 308ms/step - loss: 552.3709 - mean_squared_error_1e6: 552.3710 - psnr: 58310.8317 - val_loss: 179.5501 - val_mean_squared_error_1e6: 179.5501 - val_psnr: 59309.8651

Epoch 00314: val_psnr did not improve from 60014.82957
Epoch 315/400
249/249 [==============================] - 77s 308ms/step - loss: 446.8232 - mean_squared_error_1e6: 446.8233 - psnr: 58706.7832 - val_loss: 193.0907 - val_mean_squared_error_1e6: 193.0907 - val_psnr: 59003.4808

Epoch 00315: val_psnr did not improve from 60014.82957
Epoch 316/400
249/249 [==============================] - 77s 308ms/step - loss: 617.3842 - mean_squared_error_1e6: 617.3843 - psnr: 58202.1119 - val_loss: 155.5815 - va

249/249 [==============================] - 77s 308ms/step - loss: 659.5544 - mean_squared_error_1e6: 659.5544 - psnr: 58627.3224 - val_loss: 150.6852 - val_mean_squared_error_1e6: 150.6852 - val_psnr: 59277.5912

Epoch 00371: val_psnr did not improve from 60014.82957
Epoch 372/400
249/249 [==============================] - 77s 308ms/step - loss: 600.6213 - mean_squared_error_1e6: 600.6214 - psnr: 58314.1957 - val_loss: 157.6611 - val_mean_squared_error_1e6: 157.6611 - val_psnr: 59498.8123

Epoch 00372: val_psnr did not improve from 60014.82957
Epoch 373/400
249/249 [==============================] - 77s 308ms/step - loss: 590.8291 - mean_squared_error_1e6: 590.8291 - psnr: 58541.5968 - val_loss: 153.7353 - val_mean_squared_error_1e6: 153.7353 - val_psnr: 59073.8722

Epoch 00373: val_psnr did not improve from 60014.82957
Epoch 374/400
249/249 [==============================] - 77s 308ms/step - loss: 466.4606 - mean_squared_error_1e6: 466.4607 - psnr: 58906.7056 - val_loss: 149.3451 - va

In [9]:
depth = 4
n_fliter = 64
epochs = 400
loss = 'mse1e6'
optimizer = 'Adam'
batch_size = 5
model_type = 'PET-only' # 'PET-only', 'PET-MR', 'MR-only'
LOOCV = 12 # [0,19]

train_a_UNet(depth=depth, n_fliter=n_fliter,
             epochs=epochs, loss=loss,
             optimizer=optimizer, batch_size=batch_size,
             LOOCV=LOOCV, model_type=model_type)
print("Process is finished for "+model_type + '_' + str(LOOCV))

Epoch 1/400
249/249 [==============================] - 83s 333ms/step - loss: 13255.5566 - mean_squared_error_1e6: 13255.5570 - psnr: 44077.3433 - val_loss: 1884.6980 - val_mean_squared_error_1e6: 1884.6980 - val_psnr: 47840.2917

Epoch 00001: val_psnr improved from -inf to 47840.29165, saving model to .\training_models\PET-only_12\model.hdf5
Epoch 2/400
249/249 [==============================] - 79s 319ms/step - loss: 2401.6478 - mean_squared_error_1e6: 2401.6478 - psnr: 47376.2480 - val_loss: 1304.7163 - val_mean_squared_error_1e6: 1304.7163 - val_psnr: 49286.8766

Epoch 00002: val_psnr improved from 47840.29165 to 49286.87658, saving model to .\training_models\PET-only_12\model.hdf5
Epoch 3/400
249/249 [==============================] - 79s 316ms/step - loss: 1879.8913 - mean_squared_error_1e6: 1879.8913 - psnr: 48442.5869 - val_loss: 1038.1803 - val_mean_squared_error_1e6: 1038.1803 - val_psnr: 50403.3680

Epoch 00003: val_psnr improved from 49286.87658 to 50403.36798, saving model

249/249 [==============================] - 78s 312ms/step - loss: 695.8575 - mean_squared_error_1e6: 695.8577 - psnr: 57488.8078 - val_loss: 143.3591 - val_mean_squared_error_1e6: 143.3591 - val_psnr: 59516.0856

Epoch 00053: val_psnr did not improve from 59683.49070
Epoch 54/400
249/249 [==============================] - 77s 311ms/step - loss: 560.1521 - mean_squared_error_1e6: 560.1522 - psnr: 57706.5426 - val_loss: 138.7035 - val_mean_squared_error_1e6: 138.7035 - val_psnr: 59819.0572

Epoch 00054: val_psnr improved from 59683.49070 to 59819.05719, saving model to .\training_models\PET-only_12\model.hdf5
Epoch 55/400
249/249 [==============================] - 77s 310ms/step - loss: 617.1069 - mean_squared_error_1e6: 617.1070 - psnr: 57669.5852 - val_loss: 150.5353 - val_mean_squared_error_1e6: 150.5353 - val_psnr: 59515.4053

Epoch 00055: val_psnr did not improve from 59819.05719
Epoch 56/400
249/249 [==============================] - 77s 311ms/step - loss: 674.3852 - mean_squared_e

Epoch 82/400
249/249 [==============================] - 78s 312ms/step - loss: 425.0044 - mean_squared_error_1e6: 425.0045 - psnr: 58517.6265 - val_loss: 143.2114 - val_mean_squared_error_1e6: 143.2114 - val_psnr: 59448.0861

Epoch 00082: val_psnr did not improve from 59819.05719
Epoch 83/400
249/249 [==============================] - 78s 313ms/step - loss: 760.8263 - mean_squared_error_1e6: 760.8265 - psnr: 56989.4556 - val_loss: 176.3478 - val_mean_squared_error_1e6: 176.3478 - val_psnr: 59083.4650

Epoch 00083: val_psnr did not improve from 59819.05719
Epoch 84/400
249/249 [==============================] - 78s 312ms/step - loss: 481.8964 - mean_squared_error_1e6: 481.8965 - psnr: 58297.5930 - val_loss: 136.9848 - val_mean_squared_error_1e6: 136.9848 - val_psnr: 59688.5639

Epoch 00084: val_psnr did not improve from 59819.05719
Epoch 85/400
249/249 [==============================] - 77s 309ms/step - loss: 717.4143 - mean_squared_error_1e6: 717.4145 - psnr: 57542.3232 - val_loss: 147

Epoch 111/400
249/249 [==============================] - 77s 309ms/step - loss: 527.9722 - mean_squared_error_1e6: 527.9723 - psnr: 58046.6335 - val_loss: 161.7672 - val_mean_squared_error_1e6: 161.7672 - val_psnr: 59442.4217

Epoch 00111: val_psnr did not improve from 59945.66455
Epoch 112/400
249/249 [==============================] - 77s 310ms/step - loss: 613.8839 - mean_squared_error_1e6: 613.8840 - psnr: 57481.2746 - val_loss: 138.6875 - val_mean_squared_error_1e6: 138.6875 - val_psnr: 59489.3637

Epoch 00112: val_psnr did not improve from 59945.66455
Epoch 113/400
249/249 [==============================] - 77s 310ms/step - loss: 657.9333 - mean_squared_error_1e6: 657.9334 - psnr: 57705.8351 - val_loss: 166.6812 - val_mean_squared_error_1e6: 166.6812 - val_psnr: 59113.7509

Epoch 00113: val_psnr did not improve from 59945.66455
Epoch 114/400
249/249 [==============================] - 77s 310ms/step - loss: 524.7541 - mean_squared_error_1e6: 524.7542 - psnr: 58073.2073 - val_loss:

249/249 [==============================] - 77s 309ms/step - loss: 530.6941 - mean_squared_error_1e6: 530.6942 - psnr: 57968.0501 - val_loss: 171.3647 - val_mean_squared_error_1e6: 171.3647 - val_psnr: 59521.2252

Epoch 00169: val_psnr did not improve from 59945.66455
Epoch 170/400
249/249 [==============================] - 77s 309ms/step - loss: 566.1562 - mean_squared_error_1e6: 566.1563 - psnr: 58479.1038 - val_loss: 146.1132 - val_mean_squared_error_1e6: 146.1132 - val_psnr: 59496.7851

Epoch 00170: val_psnr did not improve from 59945.66455
Epoch 171/400
249/249 [==============================] - 77s 308ms/step - loss: 673.7071 - mean_squared_error_1e6: 673.7072 - psnr: 57894.2151 - val_loss: 151.6651 - val_mean_squared_error_1e6: 151.6651 - val_psnr: 59242.1267

Epoch 00171: val_psnr did not improve from 59945.66455
Epoch 172/400
249/249 [==============================] - 77s 308ms/step - loss: 582.5213 - mean_squared_error_1e6: 582.5214 - psnr: 58389.9925 - val_loss: 148.8485 - va

249/249 [==============================] - 77s 309ms/step - loss: 576.2996 - mean_squared_error_1e6: 576.2997 - psnr: 58336.8307 - val_loss: 139.0333 - val_mean_squared_error_1e6: 139.0333 - val_psnr: 59523.9520

Epoch 00227: val_psnr did not improve from 59945.66455
Epoch 228/400
249/249 [==============================] - 77s 309ms/step - loss: 675.4833 - mean_squared_error_1e6: 675.4833 - psnr: 58298.1095 - val_loss: 159.8033 - val_mean_squared_error_1e6: 159.8033 - val_psnr: 59091.9853

Epoch 00228: val_psnr did not improve from 59945.66455
Epoch 229/400
249/249 [==============================] - 77s 309ms/step - loss: 633.9446 - mean_squared_error_1e6: 633.9447 - psnr: 58345.5334 - val_loss: 140.7610 - val_mean_squared_error_1e6: 140.7610 - val_psnr: 59639.9852

Epoch 00229: val_psnr did not improve from 59945.66455
Epoch 230/400
249/249 [==============================] - 77s 308ms/step - loss: 648.9841 - mean_squared_error_1e6: 648.9841 - psnr: 58149.9277 - val_loss: 160.7188 - va

249/249 [==============================] - 77s 309ms/step - loss: 644.8900 - mean_squared_error_1e6: 644.8901 - psnr: 58411.2651 - val_loss: 139.1667 - val_mean_squared_error_1e6: 139.1667 - val_psnr: 59733.9213

Epoch 00285: val_psnr did not improve from 59945.66455
Epoch 286/400
249/249 [==============================] - 77s 308ms/step - loss: 554.7442 - mean_squared_error_1e6: 554.7443 - psnr: 58432.5532 - val_loss: 139.1212 - val_mean_squared_error_1e6: 139.1212 - val_psnr: 59651.6888

Epoch 00286: val_psnr did not improve from 59945.66455
Epoch 287/400
249/249 [==============================] - 77s 308ms/step - loss: 601.5546 - mean_squared_error_1e6: 601.5547 - psnr: 57933.9397 - val_loss: 164.8768 - val_mean_squared_error_1e6: 164.8768 - val_psnr: 59133.2392

Epoch 00287: val_psnr did not improve from 59945.66455
Epoch 288/400
249/249 [==============================] - 77s 308ms/step - loss: 599.3751 - mean_squared_error_1e6: 599.3751 - psnr: 58478.3093 - val_loss: 142.6515 - va


Epoch 00313: val_psnr did not improve from 60177.21831
Epoch 314/400
249/249 [==============================] - 77s 308ms/step - loss: 540.1408 - mean_squared_error_1e6: 540.1408 - psnr: 58443.9552 - val_loss: 143.1516 - val_mean_squared_error_1e6: 143.1516 - val_psnr: 59723.9654

Epoch 00314: val_psnr did not improve from 60177.21831
Epoch 315/400
249/249 [==============================] - 77s 308ms/step - loss: 441.2902 - mean_squared_error_1e6: 441.2902 - psnr: 58752.3740 - val_loss: 148.5695 - val_mean_squared_error_1e6: 148.5695 - val_psnr: 59650.9674

Epoch 00315: val_psnr did not improve from 60177.21831
Epoch 316/400
249/249 [==============================] - 77s 308ms/step - loss: 638.1934 - mean_squared_error_1e6: 638.1934 - psnr: 57987.2854 - val_loss: 162.3978 - val_mean_squared_error_1e6: 162.3978 - val_psnr: 59305.3077

Epoch 00316: val_psnr did not improve from 60177.21831
Epoch 317/400
249/249 [==============================] - 77s 309ms/step - loss: 535.7656 - mean_sq

Epoch 343/400
249/249 [==============================] - 77s 308ms/step - loss: 579.9852 - mean_squared_error_1e6: 579.9853 - psnr: 58294.4322 - val_loss: 150.7436 - val_mean_squared_error_1e6: 150.7436 - val_psnr: 59499.0549

Epoch 00343: val_psnr did not improve from 60177.21831
Epoch 344/400
249/249 [==============================] - 77s 308ms/step - loss: 552.8877 - mean_squared_error_1e6: 552.8877 - psnr: 58472.3564 - val_loss: 157.9495 - val_mean_squared_error_1e6: 157.9495 - val_psnr: 59499.1145

Epoch 00344: val_psnr did not improve from 60177.21831
Epoch 345/400
249/249 [==============================] - 77s 309ms/step - loss: 537.5775 - mean_squared_error_1e6: 537.5776 - psnr: 58391.3970 - val_loss: 174.6305 - val_mean_squared_error_1e6: 174.6306 - val_psnr: 59276.9565

Epoch 00345: val_psnr did not improve from 60177.21831
Epoch 346/400
249/249 [==============================] - 77s 308ms/step - loss: 659.8342 - mean_squared_error_1e6: 659.8343 - psnr: 58210.4125 - val_loss:

Prediction!
Process is finished for PET-only_12


In [9]:
depth = 4
n_fliter = 64
epochs = 400
loss = 'mse1e6'
optimizer = 'Adam'
batch_size = 5
model_type = 'PET-only' # 'PET-only', 'PET-MR', 'MR-only'
LOOCV = 11 # [0,19]

train_a_UNet(depth=depth, n_fliter=n_fliter,
             epochs=epochs, loss=loss,
             optimizer=optimizer, batch_size=batch_size,
             LOOCV=LOOCV, model_type=model_type)
print("Process is finished for "+model_type + '_' + str(LOOCV))

Epoch 1/400
249/249 [==============================] - 82s 329ms/step - loss: 13311.5956 - mean_squared_error_1e6: 13311.5959 - psnr: 44019.0044 - val_loss: 1830.5313 - val_mean_squared_error_1e6: 1830.5313 - val_psnr: 47909.1843

Epoch 00001: val_psnr improved from -inf to 47909.18430, saving model to .\training_models\PET-only_11\model.hdf5
Epoch 2/400
249/249 [==============================] - 80s 320ms/step - loss: 2431.9712 - mean_squared_error_1e6: 2431.9711 - psnr: 47294.0156 - val_loss: 1308.3195 - val_mean_squared_error_1e6: 1308.3196 - val_psnr: 49257.5376

Epoch 00002: val_psnr improved from 47909.18430 to 49257.53761, saving model to .\training_models\PET-only_11\model.hdf5
Epoch 3/400
249/249 [==============================] - 78s 314ms/step - loss: 1908.4650 - mean_squared_error_1e6: 1908.4650 - psnr: 48348.2195 - val_loss: 1022.1579 - val_mean_squared_error_1e6: 1022.1579 - val_psnr: 50465.9659

Epoch 00003: val_psnr improved from 49257.53761 to 50465.96594, saving model

249/249 [==============================] - 80s 320ms/step - loss: 699.5519 - mean_squared_error_1e6: 699.5521 - psnr: 57386.9695 - val_loss: 145.4279 - val_mean_squared_error_1e6: 145.4279 - val_psnr: 59446.7708

Epoch 00053: val_psnr did not improve from 59538.98266
Epoch 54/400
249/249 [==============================] - 80s 320ms/step - loss: 567.1849 - mean_squared_error_1e6: 567.1850 - psnr: 57573.7825 - val_loss: 146.8635 - val_mean_squared_error_1e6: 146.8635 - val_psnr: 59702.4688

Epoch 00054: val_psnr improved from 59538.98266 to 59702.46875, saving model to .\training_models\PET-only_11\model.hdf5
Epoch 55/400
249/249 [==============================] - 80s 320ms/step - loss: 617.9058 - mean_squared_error_1e6: 617.9059 - psnr: 57638.6189 - val_loss: 146.9551 - val_mean_squared_error_1e6: 146.9551 - val_psnr: 59633.6365

Epoch 00055: val_psnr did not improve from 59702.46875
Epoch 56/400
249/249 [==============================] - 80s 319ms/step - loss: 689.3301 - mean_squared_e


Epoch 00109: val_psnr did not improve from 59964.93201
Epoch 110/400
249/249 [==============================] - 78s 312ms/step - loss: 771.0034 - mean_squared_error_1e6: 771.0036 - psnr: 57352.6211 - val_loss: 159.7632 - val_mean_squared_error_1e6: 159.7632 - val_psnr: 59074.2214

Epoch 00110: val_psnr did not improve from 59964.93201
Epoch 111/400
249/249 [==============================] - 78s 312ms/step - loss: 534.8218 - mean_squared_error_1e6: 534.8219 - psnr: 57876.8682 - val_loss: 157.5243 - val_mean_squared_error_1e6: 157.5243 - val_psnr: 59616.4563

Epoch 00111: val_psnr did not improve from 59964.93201
Epoch 112/400
249/249 [==============================] - 78s 312ms/step - loss: 610.8311 - mean_squared_error_1e6: 610.8312 - psnr: 57531.9019 - val_loss: 132.8289 - val_mean_squared_error_1e6: 132.8289 - val_psnr: 59717.0148

Epoch 00112: val_psnr did not improve from 59964.93201
Epoch 113/400
249/249 [==============================] - 78s 315ms/step - loss: 634.0687 - mean_sq

Epoch 139/400
249/249 [==============================] - 79s 317ms/step - loss: 637.3800 - mean_squared_error_1e6: 637.3801 - psnr: 58077.8247 - val_loss: 141.9222 - val_mean_squared_error_1e6: 141.9222 - val_psnr: 59601.1447

Epoch 00139: val_psnr did not improve from 59964.93201
Epoch 140/400
249/249 [==============================] - 79s 316ms/step - loss: 534.8429 - mean_squared_error_1e6: 534.8430 - psnr: 58348.2231 - val_loss: 153.9879 - val_mean_squared_error_1e6: 153.9879 - val_psnr: 59842.7410

Epoch 00140: val_psnr did not improve from 59964.93201
Epoch 141/400
249/249 [==============================] - 79s 316ms/step - loss: 624.0560 - mean_squared_error_1e6: 624.0561 - psnr: 57536.0092 - val_loss: 137.0475 - val_mean_squared_error_1e6: 137.0475 - val_psnr: 59572.0154

Epoch 00141: val_psnr did not improve from 59964.93201
Epoch 142/400
249/249 [==============================] - 79s 316ms/step - loss: 626.6531 - mean_squared_error_1e6: 626.6532 - psnr: 57752.8120 - val_loss:

249/249 [==============================] - 77s 308ms/step - loss: 630.7815 - mean_squared_error_1e6: 630.7815 - psnr: 58102.2506 - val_loss: 151.8144 - val_mean_squared_error_1e6: 151.8144 - val_psnr: 59270.2695

Epoch 00197: val_psnr did not improve from 59964.93201
Epoch 198/400
249/249 [==============================] - 77s 309ms/step - loss: 601.8320 - mean_squared_error_1e6: 601.8321 - psnr: 58222.8049 - val_loss: 139.7090 - val_mean_squared_error_1e6: 139.7090 - val_psnr: 59852.4713

Epoch 00198: val_psnr did not improve from 59964.93201
Epoch 199/400
249/249 [==============================] - 77s 308ms/step - loss: 510.6399 - mean_squared_error_1e6: 510.6400 - psnr: 58404.8252 - val_loss: 160.9344 - val_mean_squared_error_1e6: 160.9344 - val_psnr: 59243.6179

Epoch 00199: val_psnr did not improve from 59964.93201
Epoch 200/400
249/249 [==============================] - 77s 308ms/step - loss: 752.9388 - mean_squared_error_1e6: 752.9389 - psnr: 57856.6529 - val_loss: 164.6515 - va

249/249 [==============================] - 77s 309ms/step - loss: 556.4958 - mean_squared_error_1e6: 556.4959 - psnr: 58240.4682 - val_loss: 167.9472 - val_mean_squared_error_1e6: 167.9472 - val_psnr: 58916.7717

Epoch 00255: val_psnr did not improve from 59964.93201
Epoch 256/400
249/249 [==============================] - 77s 309ms/step - loss: 611.4563 - mean_squared_error_1e6: 611.4565 - psnr: 58261.6020 - val_loss: 157.0067 - val_mean_squared_error_1e6: 157.0067 - val_psnr: 59154.6142

Epoch 00256: val_psnr did not improve from 59964.93201
Epoch 257/400
249/249 [==============================] - 77s 309ms/step - loss: 586.9129 - mean_squared_error_1e6: 586.9129 - psnr: 58378.7413 - val_loss: 164.6495 - val_mean_squared_error_1e6: 164.6495 - val_psnr: 58910.5402

Epoch 00257: val_psnr did not improve from 59964.93201
Epoch 258/400
249/249 [==============================] - 77s 308ms/step - loss: 617.5142 - mean_squared_error_1e6: 617.5143 - psnr: 58134.6345 - val_loss: 165.8451 - va

249/249 [==============================] - 77s 308ms/step - loss: 594.2295 - mean_squared_error_1e6: 594.2295 - psnr: 58342.5498 - val_loss: 220.3916 - val_mean_squared_error_1e6: 220.3916 - val_psnr: 59234.4881

Epoch 00313: val_psnr did not improve from 60095.94917
Epoch 314/400
249/249 [==============================] - 77s 308ms/step - loss: 541.3636 - mean_squared_error_1e6: 541.3636 - psnr: 58418.2795 - val_loss: 155.2727 - val_mean_squared_error_1e6: 155.2727 - val_psnr: 59130.7098

Epoch 00314: val_psnr did not improve from 60095.94917
Epoch 315/400
249/249 [==============================] - 77s 308ms/step - loss: 448.0224 - mean_squared_error_1e6: 448.0225 - psnr: 58666.2430 - val_loss: 202.8586 - val_mean_squared_error_1e6: 202.8586 - val_psnr: 58600.3562

Epoch 00315: val_psnr did not improve from 60095.94917
Epoch 316/400
249/249 [==============================] - 77s 308ms/step - loss: 619.0927 - mean_squared_error_1e6: 619.0927 - psnr: 58155.0954 - val_loss: 162.1271 - va

249/249 [==============================] - 77s 308ms/step - loss: 668.2183 - mean_squared_error_1e6: 668.2184 - psnr: 58466.9950 - val_loss: 210.4964 - val_mean_squared_error_1e6: 210.4964 - val_psnr: 58073.5783

Epoch 00371: val_psnr did not improve from 60095.94917
Epoch 372/400
249/249 [==============================] - 77s 308ms/step - loss: 559.3324 - mean_squared_error_1e6: 559.3325 - psnr: 58725.9058 - val_loss: 156.3493 - val_mean_squared_error_1e6: 156.3493 - val_psnr: 59075.2211

Epoch 00372: val_psnr did not improve from 60095.94917
Epoch 373/400
249/249 [==============================] - 77s 309ms/step - loss: 592.2769 - mean_squared_error_1e6: 592.2770 - psnr: 58486.2902 - val_loss: 155.0872 - val_mean_squared_error_1e6: 155.0872 - val_psnr: 59046.3594

Epoch 00373: val_psnr did not improve from 60095.94917
Epoch 374/400
249/249 [==============================] - 77s 309ms/step - loss: 469.0301 - mean_squared_error_1e6: 469.0302 - psnr: 58818.4133 - val_loss: 154.1584 - va

In [9]:
depth = 4
n_fliter = 64
epochs = 400
loss = 'mse1e6'
optimizer = 'Adam'
batch_size = 5
model_type = 'PET-only' # 'PET-only', 'PET-MR', 'MR-only'
LOOCV = 10 # [0,19]

train_a_UNet(depth=depth, n_fliter=n_fliter,
             epochs=epochs, loss=loss,
             optimizer=optimizer, batch_size=batch_size,
             LOOCV=LOOCV, model_type=model_type)
print("Process is finished for "+model_type + '_' + str(LOOCV))

Epoch 1/400
249/249 [==============================] - 82s 331ms/step - loss: 13383.0874 - mean_squared_error_1e6: 13383.0875 - psnr: 44074.8027 - val_loss: 1923.0190 - val_mean_squared_error_1e6: 1923.0190 - val_psnr: 47784.0063

Epoch 00001: val_psnr improved from -inf to 47784.00632, saving model to .\training_models\PET-only_10\model.hdf5
Epoch 2/400
249/249 [==============================] - 79s 317ms/step - loss: 2406.5896 - mean_squared_error_1e6: 2406.5896 - psnr: 47348.9871 - val_loss: 1323.3706 - val_mean_squared_error_1e6: 1323.3706 - val_psnr: 49204.1466

Epoch 00002: val_psnr improved from 47784.00632 to 49204.14659, saving model to .\training_models\PET-only_10\model.hdf5
Epoch 3/400
249/249 [==============================] - 79s 316ms/step - loss: 1887.7676 - mean_squared_error_1e6: 1887.7677 - psnr: 48426.6615 - val_loss: 1035.1341 - val_mean_squared_error_1e6: 1035.1341 - val_psnr: 50415.5557

Epoch 00003: val_psnr improved from 49204.14659 to 50415.55570, saving model


Epoch 00052: val_psnr improved from 59600.52317 to 59702.31575, saving model to .\training_models\PET-only_10\model.hdf5
Epoch 53/400
249/249 [==============================] - 79s 316ms/step - loss: 677.8225 - mean_squared_error_1e6: 677.8227 - psnr: 57797.3631 - val_loss: 142.4219 - val_mean_squared_error_1e6: 142.4219 - val_psnr: 59501.4345

Epoch 00053: val_psnr did not improve from 59702.31575
Epoch 54/400
249/249 [==============================] - 79s 317ms/step - loss: 555.5340 - mean_squared_error_1e6: 555.5341 - psnr: 57770.0630 - val_loss: 152.4265 - val_mean_squared_error_1e6: 152.4265 - val_psnr: 59601.8771

Epoch 00054: val_psnr did not improve from 59702.31575
Epoch 55/400
249/249 [==============================] - 79s 316ms/step - loss: 626.5291 - mean_squared_error_1e6: 626.5292 - psnr: 57628.4396 - val_loss: 157.1593 - val_mean_squared_error_1e6: 157.1593 - val_psnr: 59440.5123

Epoch 00055: val_psnr did not improve from 59702.31575
Epoch 56/400
249/249 [=============

249/249 [==============================] - 77s 309ms/step - loss: 551.2206 - mean_squared_error_1e6: 551.2208 - psnr: 58079.1104 - val_loss: 160.5148 - val_mean_squared_error_1e6: 160.5148 - val_psnr: 59423.4134

Epoch 00081: val_psnr did not improve from 59901.92021
Epoch 82/400
249/249 [==============================] - 77s 309ms/step - loss: 429.5754 - mean_squared_error_1e6: 429.5755 - psnr: 58608.5061 - val_loss: 156.9194 - val_mean_squared_error_1e6: 156.9194 - val_psnr: 59614.6519

Epoch 00082: val_psnr did not improve from 59901.92021
Epoch 83/400
249/249 [==============================] - 77s 309ms/step - loss: 734.1211 - mean_squared_error_1e6: 734.1212 - psnr: 57415.7146 - val_loss: 146.0768 - val_mean_squared_error_1e6: 146.0768 - val_psnr: 59444.7619

Epoch 00083: val_psnr did not improve from 59901.92021
Epoch 84/400
249/249 [==============================] - 77s 309ms/step - loss: 470.7070 - mean_squared_error_1e6: 470.7071 - psnr: 58538.5938 - val_loss: 135.8365 - val_m

249/249 [==============================] - 77s 309ms/step - loss: 790.4828 - mean_squared_error_1e6: 790.4829 - psnr: 57394.9466 - val_loss: 183.1874 - val_mean_squared_error_1e6: 183.1874 - val_psnr: 58736.8832

Epoch 00110: val_psnr did not improve from 59920.77238
Epoch 111/400
249/249 [==============================] - 77s 308ms/step - loss: 516.7474 - mean_squared_error_1e6: 516.7475 - psnr: 58297.8530 - val_loss: 164.9078 - val_mean_squared_error_1e6: 164.9078 - val_psnr: 59392.4831

Epoch 00111: val_psnr did not improve from 59920.77238
Epoch 112/400
249/249 [==============================] - 77s 309ms/step - loss: 597.8481 - mean_squared_error_1e6: 597.8482 - psnr: 57743.3895 - val_loss: 136.4530 - val_mean_squared_error_1e6: 136.4530 - val_psnr: 59516.4747

Epoch 00112: val_psnr did not improve from 59920.77238
Epoch 113/400
249/249 [==============================] - 77s 309ms/step - loss: 622.8643 - mean_squared_error_1e6: 622.8644 - psnr: 58086.9936 - val_loss: 152.2783 - va

249/249 [==============================] - 77s 308ms/step - loss: 650.1274 - mean_squared_error_1e6: 650.1275 - psnr: 57752.5567 - val_loss: 152.2110 - val_mean_squared_error_1e6: 152.2110 - val_psnr: 59245.5875

Epoch 00168: val_psnr did not improve from 59920.77238
Epoch 169/400
249/249 [==============================] - 77s 309ms/step - loss: 498.3998 - mean_squared_error_1e6: 498.3999 - psnr: 58406.7011 - val_loss: 139.0600 - val_mean_squared_error_1e6: 139.0600 - val_psnr: 59736.9267

Epoch 00169: val_psnr did not improve from 59920.77238
Epoch 170/400
249/249 [==============================] - 77s 308ms/step - loss: 557.5923 - mean_squared_error_1e6: 557.5924 - psnr: 58758.9418 - val_loss: 146.9119 - val_mean_squared_error_1e6: 146.9119 - val_psnr: 59429.0083

Epoch 00170: val_psnr did not improve from 59920.77238
Epoch 171/400
249/249 [==============================] - 77s 308ms/step - loss: 664.9909 - mean_squared_error_1e6: 664.9910 - psnr: 58125.3855 - val_loss: 159.5157 - va

249/249 [==============================] - 77s 309ms/step - loss: 617.1283 - mean_squared_error_1e6: 617.1284 - psnr: 58528.0161 - val_loss: 147.6389 - val_mean_squared_error_1e6: 147.6389 - val_psnr: 59362.1248

Epoch 00226: val_psnr did not improve from 59944.98881
Epoch 227/400
249/249 [==============================] - 77s 309ms/step - loss: 579.9691 - mean_squared_error_1e6: 579.9692 - psnr: 58460.7107 - val_loss: 161.7437 - val_mean_squared_error_1e6: 161.7437 - val_psnr: 59349.6002

Epoch 00227: val_psnr did not improve from 59944.98881
Epoch 228/400
249/249 [==============================] - 77s 309ms/step - loss: 673.1856 - mean_squared_error_1e6: 673.1856 - psnr: 58390.8722 - val_loss: 167.9409 - val_mean_squared_error_1e6: 167.9409 - val_psnr: 58970.6232

Epoch 00228: val_psnr did not improve from 59944.98881
Epoch 229/400
249/249 [==============================] - 77s 309ms/step - loss: 633.5057 - mean_squared_error_1e6: 633.5057 - psnr: 58446.4283 - val_loss: 142.4746 - va

249/249 [==============================] - 77s 309ms/step - loss: 604.6731 - mean_squared_error_1e6: 604.6731 - psnr: 58320.2318 - val_loss: 171.8306 - val_mean_squared_error_1e6: 171.8306 - val_psnr: 59462.7054

Epoch 00284: val_psnr did not improve from 59944.98881
Epoch 285/400
249/249 [==============================] - 77s 308ms/step - loss: 636.1798 - mean_squared_error_1e6: 636.1798 - psnr: 58631.8138 - val_loss: 137.0807 - val_mean_squared_error_1e6: 137.0807 - val_psnr: 59762.2739

Epoch 00285: val_psnr did not improve from 59944.98881
Epoch 286/400
249/249 [==============================] - 77s 308ms/step - loss: 553.4558 - mean_squared_error_1e6: 553.4559 - psnr: 58584.8567 - val_loss: 162.2498 - val_mean_squared_error_1e6: 162.2498 - val_psnr: 59509.9470

Epoch 00286: val_psnr did not improve from 59944.98881
Epoch 287/400
249/249 [==============================] - 77s 308ms/step - loss: 565.6128 - mean_squared_error_1e6: 565.6128 - psnr: 58446.2873 - val_loss: 162.9269 - va

Epoch 313/400
249/249 [==============================] - 77s 308ms/step - loss: 578.0949 - mean_squared_error_1e6: 578.0949 - psnr: 58703.1708 - val_loss: 133.1241 - val_mean_squared_error_1e6: 133.1241 - val_psnr: 60050.1540

Epoch 00313: val_psnr did not improve from 60315.83743
Epoch 314/400
249/249 [==============================] - 77s 308ms/step - loss: 536.2808 - mean_squared_error_1e6: 536.2808 - psnr: 58638.8205 - val_loss: 163.6610 - val_mean_squared_error_1e6: 163.6610 - val_psnr: 59806.0153

Epoch 00314: val_psnr did not improve from 60315.83743
Epoch 315/400
249/249 [==============================] - 77s 308ms/step - loss: 430.3713 - mean_squared_error_1e6: 430.3713 - psnr: 59056.8463 - val_loss: 149.2860 - val_mean_squared_error_1e6: 149.2860 - val_psnr: 59506.8521

Epoch 00315: val_psnr did not improve from 60315.83743
Epoch 316/400
249/249 [==============================] - 77s 308ms/step - loss: 608.5129 - mean_squared_error_1e6: 608.5130 - psnr: 58409.2190 - val_loss:

249/249 [==============================] - 77s 308ms/step - loss: 654.8050 - mean_squared_error_1e6: 654.8051 - psnr: 58798.5749 - val_loss: 160.6868 - val_mean_squared_error_1e6: 160.6868 - val_psnr: 59657.2422

Epoch 00371: val_psnr did not improve from 60315.83743
Epoch 372/400
249/249 [==============================] - 77s 308ms/step - loss: 550.9576 - mean_squared_error_1e6: 550.9577 - psnr: 58962.2847 - val_loss: 136.1888 - val_mean_squared_error_1e6: 136.1888 - val_psnr: 59887.1506

Epoch 00372: val_psnr did not improve from 60315.83743
Epoch 373/400
249/249 [==============================] - 77s 308ms/step - loss: 576.6174 - mean_squared_error_1e6: 576.6175 - psnr: 58771.6019 - val_loss: 142.6363 - val_mean_squared_error_1e6: 142.6363 - val_psnr: 59559.1951

Epoch 00373: val_psnr did not improve from 60315.83743
Epoch 374/400
249/249 [==============================] - 77s 308ms/step - loss: 545.8405 - mean_squared_error_1e6: 545.8405 - psnr: 58878.8569 - val_loss: 149.3504 - va